In [2]:
!pip install pettingzoo pygame numpy
!pip install moviepy pygame

192.06s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


197.94s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [3]:
from pettingzoo.sisl import waterworld_v4
env = waterworld_v4.env()

In [4]:
from tianshou.env import DummyVectorEnv
from tianshou.env.pettingzoo_env import PettingZooEnv

/home/qrbao/anaconda3/envs/rllib_o/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-16 19:13:10,830	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [5]:
def make_env():
    return PettingZooEnv(waterworld_v4.env())
k = 4  # Number of parallel environments
envs = DummyVectorEnv([make_env for _ in range(k)])  # k 个并行环境

In [6]:
# """
# Tianshou Waterworld 简化版本 - 先测试基本功能
# 每个agent使用独立的策略（从随机策略开始）

# 安装依赖：
# pip install pettingzoo[sisl]
# pip install tianshou
# pip install torch
# pip install numpy
# """

# import os
# import argparse
# import numpy as np
# import torch
# from typing import Optional, Tuple, List

# # Tianshou imports
# from tianshou.data import Collector, VectorReplayBuffer
# from tianshou.env import DummyVectorEnv
# from tianshou.env.pettingzoo_env import PettingZooEnv
# from tianshou.policy import (
#     BasePolicy, 
#     DQNPolicy, 
#     MultiAgentPolicyManager, 
#     RandomPolicy
# )
# from tianshou.trainer import offpolicy_trainer
# from tianshou.utils.net.common import Net

# # PettingZoo imports
# from pettingzoo.sisl import waterworld_v4

# def get_args():
#     """配置参数"""
#     parser = argparse.ArgumentParser()
#     parser.add_argument('--seed', type=int, default=42)
#     parser.add_argument('--buffer-size', type=int, default=20000)
#     parser.add_argument('--lr', type=float, default=1e-4)
#     parser.add_argument('--gamma', type=float, default=0.99)
#     parser.add_argument('--epoch', type=int, default=10)  # 减少训练轮数
#     parser.add_argument('--step-per-epoch', type=int, default=1000)
#     parser.add_argument('--step-per-collect', type=int, default=200)
#     parser.add_argument('--batch-size', type=int, default=64)
#     parser.add_argument('--hidden-sizes', type=int, nargs='*', default=[128, 128])
#     parser.add_argument('--training-num', type=int, default=4)
#     parser.add_argument('--test-num', type=int, default=2)
#     parser.add_argument('--device', type=str, default='cuda' if torch.cuda.is_available() else 'cpu')
#     parser.add_argument('--watch', action='store_true', default=False)
#     parser.add_argument('--render', action='store_true', default=False)
    
#     # Waterworld specific parameters
#     parser.add_argument('--n-pursuers', type=int, default=5)
    
#     return parser.parse_known_args()[0]

# def get_env(args):
#     """创建Waterworld环境"""
#     env = waterworld_v4.env(
#         n_pursuers=args.n_pursuers,
#         n_evaders=5,
#         n_poisons=10,
#         n_coop=2,
#         n_sensors=30,
#         sensor_range=0.2,
#         radius=0.015,
#         pursuer_max_accel=0.01,
#         evader_speed=0.01,
#         poison_speed=0.01,
#         poison_reward=-1.0,
#         food_reward=10.0,
#         encounter_reward=0.01,
#         thrust_penalty=-0.5,
#         local_ratio=1.0,
#         speed_features=True,
#         max_cycles=500,
#         render_mode="human" if args.render else None
#     )
#     return PettingZooEnv(env)

# def create_simple_policies(args, env) -> Tuple[List[BasePolicy], List]:
#     """
#     创建简单的策略组合：
#     - 一些使用DQN（离散化动作）
#     - 一些使用随机策略
#     """
#     agents = env.agents
#     policies = []
#     optimizers = []
    
#     # 获取环境信息
#     observation_space = env.observation_space
#     action_space = env.action_space
    
#     state_shape = observation_space.shape or observation_space.n
#     action_shape = action_space.shape or action_space.n
    
#     print(f"状态空间维度: {state_shape}")
#     print(f"动作空间: {action_space}")
#     print(f"智能体数量: {len(agents)}")
    
#     # 为连续动作空间创建离散化版本的DQN
#     # 将连续动作 [-0.01, 0.01] x [-0.01, 0.01] 离散化为9个动作
#     discrete_actions = [
#         [-0.01, -0.01], [-0.01, 0.0], [-0.01, 0.01],  # 左上, 左, 左下
#         [0.0, -0.01],   [0.0, 0.0],   [0.0, 0.01],    # 上, 停止, 下  
#         [0.01, -0.01],  [0.01, 0.0],  [0.01, 0.01]    # 右上, 右, 右下
#     ]
    
#     # 为每个agent分配策略
#     algorithm_assignment = {
#         'pursuer_0': 'DQN',      # 使用DQN
#         'pursuer_1': 'DQN',      # 使用DQN
#         'pursuer_2': 'Random',   # 随机策略
#         'pursuer_3': 'Random',   # 随机策略
#         'pursuer_4': 'DQN',      # 使用DQN
#     }
    
#     for i, agent_id in enumerate(agents):
#         algo = algorithm_assignment.get(agent_id, 'Random')
#         print(f"Agent {agent_id} 使用算法: {algo}")
        
#         if algo == 'DQN':
#             # 创建DQN网络
#             net = Net(
#                 state_shape=state_shape,
#                 action_shape=len(discrete_actions),  # 9个离散动作
#                 hidden_sizes=args.hidden_sizes,
#                 device=args.device
#             ).to(args.device)
            
#             optim = torch.optim.Adam(net.parameters(), lr=args.lr)
            
#             # 创建DQN策略
#             policy = DQNPolicy(
#                 model=net,
#                 optim=optim,
#                 discount_factor=args.gamma,
#                 estimation_step=3,
#                 target_update_freq=320
#             )
            
#             # 为DQN策略添加动作转换函数
#             policy.discrete_actions = discrete_actions
#             original_forward = policy.forward
            
#             def dqn_forward_with_continuous_output(batch, state=None, **kwargs):
#                 # 调用原始forward得到离散动作
#                 result = original_forward(batch, state, **kwargs)
                
#                 # 将离散动作索引转换为连续动作
#                 if hasattr(result, 'act'):
#                     discrete_indices = result.act
#                     continuous_actions = []
                    
#                     for idx in discrete_indices:
#                         if isinstance(idx, torch.Tensor):
#                             idx = idx.item()
#                         continuous_actions.append(discrete_actions[idx])
                    
#                     result.act = np.array(continuous_actions)
                
#                 return result
            
#             policy.forward = dqn_forward_with_continuous_output
#             optimizers.append(optim)
            
#         else:  # Random policy
#             policy = RandomPolicy(action_space)
#             optimizers.append(None)
        
#         policies.append(policy)
    
#     return policies, optimizers

# def train_agents(args):
#     """训练多智能体"""
#     print("初始化环境...")
    
#     # 环境设置
#     env = get_env(args)
#     train_envs = DummyVectorEnv([lambda: get_env(args) for _ in range(args.training_num)])
#     test_envs = DummyVectorEnv([lambda: get_env(args) for _ in range(args.test_num)])
    
#     # 设置随机种子
#     np.random.seed(args.seed)
#     torch.manual_seed(args.seed)
#     train_envs.seed(args.seed)
#     test_envs.seed(args.seed)
    
#     print("创建策略...")
#     # 创建智能体策略
#     policies, optimizers = create_simple_policies(args, env)
    
#     # 创建多智能体策略管理器
#     policy_manager = MultiAgentPolicyManager(policies, env)
    
#     print("创建收集器...")
#     # 创建收集器
#     train_collector = Collector(
#         policy_manager,
#         train_envs,
#         VectorReplayBuffer(args.buffer_size, len(train_envs)),
#         exploration_noise=True
#     )
    
#     test_collector = Collector(
#         policy_manager, 
#         test_envs,
#         exploration_noise=True
#     )
    
#     # 预收集一些数据
#     print("预收集数据...")
#     train_collector.collect(n_step=args.batch_size * args.training_num)
    
#     # 回调函数
#     def save_best_fn(policy):
#         """保存最佳策略"""
#         save_dir = "waterworld_policies"
#         os.makedirs(save_dir, exist_ok=True)
#         for i, agent_id in enumerate(env.agents):
#             if hasattr(policy.policies[agent_id], 'state_dict'):
#                 model_path = os.path.join(save_dir, f"{agent_id}_policy.pth")
#                 torch.save(policy.policies[agent_id].state_dict(), model_path)
#         print(f"策略已保存到 {save_dir}")
    
#     def stop_fn(mean_rewards):
#         """停止条件"""
#         return mean_rewards >= 5.0
    
#     def train_fn(epoch, env_step):
#         """训练时设置探索率"""
#         for agent_id in env.agents:
#             if hasattr(policy_manager.policies[agent_id], 'set_eps'):
#                 policy_manager.policies[agent_id].set_eps(0.1)
    
#     def test_fn(epoch, env_step):
#         """测试时设置探索率"""
#         for agent_id in env.agents:
#             if hasattr(policy_manager.policies[agent_id], 'set_eps'):
#                 policy_manager.policies[agent_id].set_eps(0.05)
    
#     def reward_metric(rews):
#         """奖励度量函数 - 返回所有agent的平均奖励"""
#         return rews.mean(axis=1)
    
#     # 检查是否有可训练的策略
#     trainable_policies = [p for p in policies if not isinstance(p, RandomPolicy)]
    
#     if trainable_policies:
#         print(f"开始训练 {len(trainable_policies)} 个可训练策略...")
        
#         # 使用off-policy训练器
#         result = offpolicy_trainer(
#             policy=policy_manager,
#             train_collector=train_collector,
#             test_collector=test_collector,
#             max_epoch=args.epoch,
#             step_per_epoch=args.step_per_epoch,
#             step_per_collect=args.step_per_collect,
#             episode_per_test=5,
#             batch_size=args.batch_size,
#             train_fn=train_fn,
#             test_fn=test_fn,
#             stop_fn=stop_fn,
#             save_best_fn=save_best_fn,
#             update_per_step=0.1,
#             test_in_train=False,
#             reward_metric=reward_metric
#         )
        
#         print("训练完成！")
#         print(f"训练结果: {result}")
#     else:
#         print("所有策略都是随机策略，运行测试...")
#         result = test_collector.collect(n_episode=5)
#         print(f"随机策略测试结果 - 平均奖励: {result['rew']}, 平均长度: {result['len']}")
    
#     return policy_manager, result

# def watch_performance(args):
#     """观察策略表现"""
#     print("观察策略表现...")
#     env = get_env(args)
#     policies, _ = create_simple_policies(args, env)
#     policy_manager = MultiAgentPolicyManager(policies, env)
    
#     # 尝试加载训练好的策略
#     save_dir = "waterworld_policies"
#     if os.path.exists(save_dir):
#         for i, agent_id in enumerate(env.agents):
#             if hasattr(policy_manager.policies[agent_id], 'load_state_dict'):
#                 model_path = os.path.join(save_dir, f"{agent_id}_policy.pth")
#                 if os.path.exists(model_path):
#                     policy_manager.policies[agent_id].load_state_dict(torch.load(model_path))
#                     print(f"加载了 {agent_id} 的策略")
    
#     # 需要使用向量化环境
#     test_envs = DummyVectorEnv([lambda: get_env(args) for _ in range(1)])
#     collector = Collector(policy_manager, test_envs)
    
#     print("运行测试episode...")
#     result = collector.collect(n_episode=3, render=args.render)
#     print(f"平均奖励: {result['rew']}")
#     print(f"平均长度: {result['len']}")

# def test_random_policies(args):
#     """测试纯随机策略作为基准"""
#     print("测试纯随机策略...")
#     env = get_env(args)
    
#     # 创建所有随机策略
#     policies = [RandomPolicy(env.action_space) for _ in env.agents]
#     policy_manager = MultiAgentPolicyManager(policies, env)
    
#     # 需要使用向量化环境
#     test_envs = DummyVectorEnv([lambda: get_env(args) for _ in range(1)])
    
#     # 运行测试
#     collector = Collector(policy_manager, test_envs)
#     result = collector.collect(n_episode=5, render=args.render)
#     print(f"随机策略基准 - 平均奖励: {result['rew']}, 平均长度: {result['len']}")
    
#     return result

# def simple_debug_test(args):
#     """最简单的debug测试"""
#     print("=== Debug测试：基础环境和策略创建 ===")
    
#     try:
#         # 1. 测试环境创建
#         print("1. 创建环境...")
#         env = get_env(args)
#         print(f"   环境创建成功: {type(env)}")
#         print(f"   智能体: {env.agents}")
#         print(f"   观察空间: {env.observation_space}")
#         print(f"   动作空间: {env.action_space}")
        
#         # 2. 测试策略创建
#         print("\n2. 创建策略...")
#         policies, optimizers = create_simple_policies(args, env)
#         print(f"   策略创建成功: {len(policies)} 个策略")
        
#         # 3. 测试策略管理器
#         print("\n3. 创建策略管理器...")
#         policy_manager = MultiAgentPolicyManager(policies, env)
#         print(f"   策略管理器创建成功: {type(policy_manager)}")
        
#         # 4. 测试向量化环境
#         print("\n4. 创建向量化环境...")
#         test_envs = DummyVectorEnv([lambda: get_env(args) for _ in range(1)])
#         print(f"   向量化环境创建成功: {type(test_envs)}")
        
#         # 5. 测试收集器
#         print("\n5. 创建收集器...")
#         collector = Collector(policy_manager, test_envs)
#         print(f"   收集器创建成功: {type(collector)}")
        
#         # 6. 测试收集一个step
#         print("\n6. 测试收集数据...")
#         result = collector.collect(n_step=10)
#         print(f"   数据收集成功: {result}")
        
#         print("\n=== 所有基础测试通过！===")
#         return True
        
#     except Exception as e:
#         print(f"\n!!! 测试失败: {e}")
#         import traceback
#         traceback.print_exc()
#         return False

# if __name__ == "__main__":
#     args = get_args()
    
#     print("=== Tianshou Waterworld 多智能体测试 ===")
#     print(f"设备: {args.device}")
#     print(f"智能体数量: {args.n_pursuers}")
    
#     # 首先运行debug测试
#     debug_success = simple_debug_test(args)
    
#     if not debug_success:
#         print("基础测试失败，请检查环境配置")
#         exit(1)
    
#     if args.watch:
#         watch_performance(args)
#     else:
#         # 首先测试随机策略
#         print("\n1. 测试随机策略基准:")
#         test_random_policies(args)
        
#         # 然后训练DQN策略
#         print("\n2. 训练DQN策略:")
#         policy_manager, result = train_agents(args)
        
#         # 训练完成后观察表现
#         if result:
#             print("\n3. 观察训练后的策略表现:")
#             args.render = True
#             watch_performance(args)

# """
# 使用说明：

# 1. 基础测试（只用随机策略）：
#    python waterworld_simple.py

# 2. 观察训练结果：
#    python waterworld_simple.py --watch --render

# 3. 自定义参数：
#    python waterworld_simple.py --epoch 20 --lr 1e-3

# 这个简化版本的特点：

# 1. 使用DQN处理连续动作空间（通过离散化）
# 2. 混合DQN和随机策略
# 3. 更强的错误处理和兼容性
# 4. 渐进式测试方法

# 如果这个版本能正常运行，我们再逐步添加更复杂的算法。
# """

In [7]:
# """
# Tianshou Waterworld 简化版本 - 先测试基本功能
# 每个agent使用独立的策略（从随机策略开始）

# 安装依赖：
# pip install pettingzoo[sisl]
# pip install tianshou
# pip install torch
# pip install numpy
# """

# import os
# import argparse
# import numpy as np
# import torch
# from typing import Optional, Tuple, List

# # Tianshou imports
# from tianshou.data import Collector, VectorReplayBuffer
# from tianshou.env import DummyVectorEnv
# from tianshou.env.pettingzoo_env import PettingZooEnv
# from tianshou.policy import (
#     BasePolicy, 
#     DQNPolicy, 
#     MultiAgentPolicyManager, 
#     RandomPolicy
# )
# from tianshou.trainer import offpolicy_trainer
# from tianshou.utils.net.common import Net

# # PettingZoo imports
# from pettingzoo.sisl import waterworld_v4

# def get_args():
#     """配置参数"""
#     parser = argparse.ArgumentParser()
#     parser.add_argument('--seed', type=int, default=42)
#     parser.add_argument('--buffer-size', type=int, default=20000)
#     parser.add_argument('--lr', type=float, default=1e-4)
#     parser.add_argument('--gamma', type=float, default=0.99)
#     parser.add_argument('--epoch', type=int, default=10)  # 减少训练轮数
#     parser.add_argument('--step-per-epoch', type=int, default=1000)
#     parser.add_argument('--step-per-collect', type=int, default=200)
#     parser.add_argument('--batch-size', type=int, default=64)
#     parser.add_argument('--hidden-sizes', type=int, nargs='*', default=[128, 128])
#     parser.add_argument('--training-num', type=int, default=4)
#     parser.add_argument('--test-num', type=int, default=2)
#     parser.add_argument('--device', type=str, default='cuda' if torch.cuda.is_available() else 'cpu')
#     parser.add_argument('--watch', action='store_true', default=False)
#     parser.add_argument('--render', action='store_true', default=False)
    
#     # Waterworld specific parameters
#     parser.add_argument('--n-pursuers', type=int, default=5)
    
#     return parser.parse_known_args()[0]

# def get_env(args):
#     """创建Waterworld环境"""
#     env = waterworld_v4.env(
#         n_pursuers=args.n_pursuers,
#         n_evaders=5,
#         n_poisons=10,
#         n_coop=2,
#         n_sensors=30,
#         sensor_range=0.2,
#         radius=0.015,
#         pursuer_max_accel=0.01,
#         evader_speed=0.01,
#         poison_speed=0.01,
#         poison_reward=-1.0,
#         food_reward=10.0,
#         encounter_reward=0.01,
#         thrust_penalty=-0.5,
#         local_ratio=1.0,
#         speed_features=True,
#         max_cycles=500,
#         render_mode="human" if args.render else None
#     )
#     return PettingZooEnv(env)

# def create_simple_policies(args, env) -> Tuple[List[BasePolicy], List]:
#     """
#     创建简单的策略组合：
#     - 一些使用DQN（离散化动作）
#     - 一些使用随机策略
#     """
#     agents = env.agents
#     policies = []
#     optimizers = []
    
#     # 获取环境信息
#     observation_space = env.observation_space
#     action_space = env.action_space
    
#     state_shape = observation_space.shape or observation_space.n
#     action_shape = action_space.shape or action_space.n
    
#     print(f"状态空间维度: {state_shape}")
#     print(f"动作空间: {action_space}")
#     print(f"智能体数量: {len(agents)}")
    
#     # 为连续动作空间创建离散化版本的DQN
#     # 将连续动作 [-0.01, 0.01] x [-0.01, 0.01] 离散化为9个动作
#     discrete_actions = [
#         [-0.01, -0.01], [-0.01, 0.0], [-0.01, 0.01],  # 左上, 左, 左下
#         [0.0, -0.01],   [0.0, 0.0],   [0.0, 0.01],    # 上, 停止, 下  
#         [0.01, -0.01],  [0.01, 0.0],  [0.01, 0.01]    # 右上, 右, 右下
#     ]
    
#     # 为每个agent分配策略
#     algorithm_assignment = {
#         'pursuer_0': 'DQN',      # 使用DQN
#         'pursuer_1': 'DQN',      # 使用DQN
#         'pursuer_2': 'Random',   # 随机策略
#         'pursuer_3': 'Random',   # 随机策略
#         'pursuer_4': 'DQN',      # 使用DQN
#     }
    
#     for i, agent_id in enumerate(agents):
#         algo = algorithm_assignment.get(agent_id, 'Random')
#         print(f"Agent {agent_id} 使用算法: {algo}")
        
#         if algo == 'DQN':
#             # 创建DQN网络
#             net = Net(
#                 state_shape=state_shape,
#                 action_shape=len(discrete_actions),  # 9个离散动作
#                 hidden_sizes=args.hidden_sizes,
#                 device=args.device
#             ).to(args.device)
            
#             optim = torch.optim.Adam(net.parameters(), lr=args.lr)
            
#             # 创建DQN策略
#             policy = DQNPolicy(
#                 model=net,
#                 optim=optim,
#                 discount_factor=args.gamma,
#                 estimation_step=3,
#                 target_update_freq=320
#             )
            
#             # 为DQN策略添加动作转换函数
#             policy.discrete_actions = discrete_actions
#             original_forward = policy.forward
            
#             def dqn_forward_with_continuous_output(batch, state=None, **kwargs):
#                 # 调用原始forward得到离散动作
#                 result = original_forward(batch, state, **kwargs)
                
#                 # 将离散动作索引转换为连续动作
#                 if hasattr(result, 'act'):
#                     discrete_indices = result.act
#                     continuous_actions = []
                    
#                     for idx in discrete_indices:
#                         if isinstance(idx, torch.Tensor):
#                             idx = idx.item()
#                         continuous_actions.append(discrete_actions[idx])
                    
#                     result.act = np.array(continuous_actions)
                
#                 return result
            
#             policy.forward = dqn_forward_with_continuous_output
#             optimizers.append(optim)
            
#         else:  # Random policy
#             policy = RandomPolicy(action_space)
#             optimizers.append(None)
        
#         policies.append(policy)
    
#     return policies, optimizers

# def train_agents(args):
#     """训练多智能体"""
#     print("初始化环境...")
    
#     # 环境设置
#     env = get_env(args)
#     train_envs = DummyVectorEnv([lambda: get_env(args) for _ in range(args.training_num)])
#     test_envs = DummyVectorEnv([lambda: get_env(args) for _ in range(args.test_num)])
    
#     # 设置随机种子
#     np.random.seed(args.seed)
#     torch.manual_seed(args.seed)
#     train_envs.seed(args.seed)
#     test_envs.seed(args.seed)
    
#     print("创建策略...")
#     # 创建智能体策略
#     policies, optimizers = create_simple_policies(args, env)
    
#     # 创建多智能体策略管理器
#     policy_manager = MultiAgentPolicyManager(policies, env)
    
#     print("创建收集器...")
#     # 创建收集器
#     train_collector = Collector(
#         policy_manager,
#         train_envs,
#         VectorReplayBuffer(args.buffer_size, len(train_envs)),
#         exploration_noise=True
#     )
    
#     test_collector = Collector(
#         policy_manager, 
#         test_envs,
#         exploration_noise=True
#     )
    
#     # 预收集一些数据
#     print("预收集数据...")
#     train_collector.collect(n_step=args.batch_size * args.training_num)
    
#     # 回调函数
#     def save_best_fn(policy):
#         """保存最佳策略"""
#         save_dir = "waterworld_policies"
#         os.makedirs(save_dir, exist_ok=True)
#         for i, agent_id in enumerate(env.agents):
#             if hasattr(policy.policies[agent_id], 'state_dict'):
#                 model_path = os.path.join(save_dir, f"{agent_id}_policy.pth")
#                 torch.save(policy.policies[agent_id].state_dict(), model_path)
#         print(f"策略已保存到 {save_dir}")
    
#     def stop_fn(mean_rewards):
#         """停止条件"""
#         return mean_rewards >= 5.0
    
#     def train_fn(epoch, env_step):
#         """训练时设置探索率"""
#         for agent_id in env.agents:
#             if hasattr(policy_manager.policies[agent_id], 'set_eps'):
#                 policy_manager.policies[agent_id].set_eps(0.1)
    
#     def test_fn(epoch, env_step):
#         """测试时设置探索率"""
#         for agent_id in env.agents:
#             if hasattr(policy_manager.policies[agent_id], 'set_eps'):
#                 policy_manager.policies[agent_id].set_eps(0.05)
    
#     def reward_metric(rews):
#         """奖励度量函数 - 返回所有agent的平均奖励"""
#         return rews.mean(axis=1)
    
#     # 检查是否有可训练的策略
#     trainable_policies = [p for p in policies if not isinstance(p, RandomPolicy)]
    
#     if trainable_policies:
#         print(f"开始训练 {len(trainable_policies)} 个可训练策略...")
        
#         # 使用off-policy训练器
#         result = offpolicy_trainer(
#             policy=policy_manager,
#             train_collector=train_collector,
#             test_collector=test_collector,
#             max_epoch=args.epoch,
#             step_per_epoch=args.step_per_epoch,
#             step_per_collect=args.step_per_collect,
#             episode_per_test=5,
#             batch_size=args.batch_size,
#             train_fn=train_fn,
#             test_fn=test_fn,
#             stop_fn=stop_fn,
#             save_best_fn=save_best_fn,
#             update_per_step=0.1,
#             test_in_train=False,
#             reward_metric=reward_metric
#         )
        
#         print("训练完成！")
#         print(f"训练结果: {result}")
#     else:
#         print("所有策略都是随机策略，运行测试...")
#         result = test_collector.collect(n_episode=5)
#         print(f"随机策略测试结果 - 平均奖励: {result['rew']}, 平均长度: {result['len']}")
    
#     return policy_manager, result

# def watch_performance(args):
#     """观察策略表现"""
#     print("观察策略表现...")
#     env = get_env(args)
#     policies, _ = create_simple_policies(args, env)
#     policy_manager = MultiAgentPolicyManager(policies, env)
    
#     # 尝试加载训练好的策略
#     save_dir = "waterworld_policies"
#     if os.path.exists(save_dir):
#         for i, agent_id in enumerate(env.agents):
#             if hasattr(policy_manager.policies[agent_id], 'load_state_dict'):
#                 model_path = os.path.join(save_dir, f"{agent_id}_policy.pth")
#                 if os.path.exists(model_path):
#                     policy_manager.policies[agent_id].load_state_dict(torch.load(model_path))
#                     print(f"加载了 {agent_id} 的策略")
    
#     # 需要使用向量化环境
#     test_envs = DummyVectorEnv([lambda: get_env(args) for _ in range(1)])
#     collector = Collector(policy_manager, test_envs)
    
#     print("运行测试episode...")
#     result = collector.collect(n_episode=3, render=args.render)
#     print(f"平均奖励: {result['rew']}")
#     print(f"平均长度: {result['len']}")

# def test_random_policies(args):
#     """测试纯随机策略作为基准"""
#     print("测试纯随机策略...")
#     env = get_env(args)
    
#     # 创建所有随机策略
#     policies = [RandomPolicy(env.action_space) for _ in env.agents]
#     policy_manager = MultiAgentPolicyManager(policies, env)
    
#     # 需要使用向量化环境
#     test_envs = DummyVectorEnv([lambda: get_env(args) for _ in range(1)])
    
#     # 运行测试
#     collector = Collector(policy_manager, test_envs)
#     result = collector.collect(n_episode=5, render=args.render)
#     print(f"随机策略基准 - 平均奖励: {result['rew']}, 平均长度: {result['len']}")
    
#     return result

# def simple_debug_test(args):
#     """最简单的debug测试"""
#     print("=== Debug测试：基础环境和策略创建 ===")
    
#     try:
#         # 1. 测试环境创建
#         print("1. 创建环境...")
#         env = get_env(args)
#         print(f"   环境创建成功: {type(env)}")
#         print(f"   智能体: {env.agents}")
#         print(f"   观察空间: {env.observation_space}")
#         print(f"   动作空间: {env.action_space}")
        
#         # 2. 测试策略创建
#         print("\n2. 创建策略...")
#         policies, optimizers = create_simple_policies(args, env)
#         print(f"   策略创建成功: {len(policies)} 个策略")
        
#         # 3. 测试策略管理器
#         print("\n3. 创建策略管理器...")
#         policy_manager = MultiAgentPolicyManager(policies, env)
#         print(f"   策略管理器创建成功: {type(policy_manager)}")
        
#         # 4. 测试向量化环境
#         print("\n4. 创建向量化环境...")
#         test_envs = DummyVectorEnv([lambda: get_env(args) for _ in range(1)])
#         print(f"   向量化环境创建成功: {type(test_envs)}")
        
#         # 5. 测试收集器
#         print("\n5. 创建收集器...")
#         collector = Collector(policy_manager, test_envs)
#         print(f"   收集器创建成功: {type(collector)}")
        
#         # 6. 测试收集一个step
#         print("\n6. 测试收集数据...")
#         result = collector.collect(n_step=10)
#         print(f"   数据收集成功: {result}")
        
#         print("\n=== 所有基础测试通过！===")
#         return True
        
#     except Exception as e:
#         print(f"\n!!! 测试失败: {e}")
#         import traceback
#         traceback.print_exc()
#         return False

# if __name__ == "__main__":
#     args = get_args()
    
#     print("=== Tianshou Waterworld 多智能体测试 ===")
#     print(f"设备: {args.device}")
#     print(f"智能体数量: {args.n_pursuers}")
    
#     # 首先运行debug测试
#     debug_success = simple_debug_test(args)
    
#     if not debug_success:
#         print("基础测试失败，请检查环境配置")
#         exit(1)
    
#     if args.watch:
#         watch_performance(args)
#     else:
#         # 首先测试随机策略
#         print("\n1. 测试随机策略基准:")
#         test_random_policies(args)
        
#         # 然后训练DQN策略
#         print("\n2. 训练DQN策略:")
#         policy_manager, result = train_agents(args)
        
#         # 训练完成后观察表现
#         if result:
#             print("\n3. 观察训练后的策略表现:")
#             args.render = True
#             watch_performance(args)

# """
# 使用说明：

# 1. 基础测试（只用随机策略）：
#    python waterworld_simple.py

# 2. 观察训练结果：
#    python waterworld_simple.py --watch --render

# 3. 自定义参数：
#    python waterworld_simple.py --epoch 20 --lr 1e-3

# 这个简化版本的特点：

# 1. 使用DQN处理连续动作空间（通过离散化）
# 2. 混合DQN和随机策略
# 3. 更强的错误处理和兼容性
# 4. 渐进式测试方法

# 如果这个版本能正常运行，我们再逐步添加更复杂的算法。
# """

In [8]:
# """
# Tianshou Waterworld 简化版本 - 先测试基本功能
# 每个agent使用独立的策略（从随机策略开始）

# 安装依赖：
# pip install pettingzoo[sisl]
# pip install tianshou
# pip install torch
# pip install numpy
# """

# import os
# import argparse
# import numpy as np
# import torch
# from typing import Optional, Tuple, List

# # Tianshou imports
# from tianshou.data import Collector, VectorReplayBuffer
# from tianshou.env import DummyVectorEnv
# from tianshou.env.pettingzoo_env import PettingZooEnv
# from tianshou.policy import (
#     BasePolicy, 
#     DQNPolicy, 
#     MultiAgentPolicyManager, 
#     RandomPolicy
# )
# from tianshou.trainer import offpolicy_trainer
# from tianshou.utils.net.common import Net

# # PettingZoo imports
# from pettingzoo.sisl import waterworld_v4

# def get_args():
#     """配置参数"""
#     parser = argparse.ArgumentParser()
#     parser.add_argument('--seed', type=int, default=42)
#     parser.add_argument('--buffer-size', type=int, default=20000)
#     parser.add_argument('--lr', type=float, default=1e-4)
#     parser.add_argument('--gamma', type=float, default=0.99)
#     parser.add_argument('--epoch', type=int, default=10)  # 减少训练轮数
#     parser.add_argument('--step-per-epoch', type=int, default=1000)
#     parser.add_argument('--step-per-collect', type=int, default=200)
#     parser.add_argument('--batch-size', type=int, default=64)
#     parser.add_argument('--hidden-sizes', type=int, nargs='*', default=[128, 128])
#     parser.add_argument('--training-num', type=int, default=4)
#     parser.add_argument('--test-num', type=int, default=2)
#     parser.add_argument('--device', type=str, default='cuda' if torch.cuda.is_available() else 'cpu')
#     parser.add_argument('--watch', action='store_true', default=False)
#     parser.add_argument('--render', action='store_true', default=False)
    
#     # Waterworld specific parameters
#     parser.add_argument('--n-pursuers', type=int, default=5)
    
#     return parser.parse_known_args()[0]

# def get_env(args):
#     """创建Waterworld环境"""
#     env = waterworld_v4.env(
#         n_pursuers=args.n_pursuers,
#         n_evaders=5,
#         n_poisons=10,
#         n_coop=2,
#         n_sensors=30,
#         sensor_range=0.2,
#         radius=0.015,
#         pursuer_max_accel=0.01,
#         evader_speed=0.01,
#         poison_speed=0.01,
#         poison_reward=-1.0,
#         food_reward=10.0,
#         encounter_reward=0.01,
#         thrust_penalty=-0.5,
#         local_ratio=1.0,
#         speed_features=True,
#         max_cycles=500,
#         render_mode="human" if args.render else None
#     )
#     return PettingZooEnv(env)

# def create_simple_policies(args, env) -> Tuple[List[BasePolicy], List]:
#     """
#     创建简单的策略组合：
#     - 一些使用DQN（离散化动作）
#     - 一些使用随机策略
#     """
#     agents = env.agents
#     policies = []
#     optimizers = []
    
#     # 获取环境信息
#     observation_space = env.observation_space
#     action_space = env.action_space
    
#     state_shape = observation_space.shape or observation_space.n
#     action_shape = action_space.shape or action_space.n
    
#     print(f"状态空间维度: {state_shape}")
#     print(f"动作空间: {action_space}")
#     print(f"智能体数量: {len(agents)}")
    
#     # 为连续动作空间创建离散化版本的DQN
#     # 将连续动作 [-0.01, 0.01] x [-0.01, 0.01] 离散化为9个动作
#     discrete_actions = [
#         [-0.01, -0.01], [-0.01, 0.0], [-0.01, 0.01],  # 左上, 左, 左下
#         [0.0, -0.01],   [0.0, 0.0],   [0.0, 0.01],    # 上, 停止, 下  
#         [0.01, -0.01],  [0.01, 0.0],  [0.01, 0.01]    # 右上, 右, 右下
#     ]
    
#     # 为每个agent分配策略
#     algorithm_assignment = {
#         'pursuer_0': 'DQN',      # 使用DQN
#         'pursuer_1': 'DQN',      # 使用DQN
#         'pursuer_2': 'Random',   # 随机策略
#         'pursuer_3': 'Random',   # 随机策略
#         'pursuer_4': 'DQN',      # 使用DQN
#     }
    
#     for i, agent_id in enumerate(agents):
#         algo = algorithm_assignment.get(agent_id, 'Random')
#         print(f"Agent {agent_id} 使用算法: {algo}")
        
#         if algo == 'DQN':
#             # 创建DQN网络
#             net = Net(
#                 state_shape=state_shape,
#                 action_shape=len(discrete_actions),  # 9个离散动作
#                 hidden_sizes=args.hidden_sizes,
#                 device=args.device
#             ).to(args.device)
            
#             optim = torch.optim.Adam(net.parameters(), lr=args.lr)
            
#             # 创建DQN策略
#             policy = DQNPolicy(
#                 model=net,
#                 optim=optim,
#                 discount_factor=args.gamma,
#                 estimation_step=3,
#                 target_update_freq=320
#             )
            
#             # 为DQN策略添加动作转换函数
#             policy.discrete_actions = discrete_actions
#             original_forward = policy.forward
            
#             def dqn_forward_with_continuous_output(batch, state=None, **kwargs):
#                 # 调用原始forward得到离散动作
#                 result = original_forward(batch, state, **kwargs)
                
#                 # 将离散动作索引转换为连续动作
#                 if hasattr(result, 'act'):
#                     discrete_indices = result.act
#                     continuous_actions = []
                    
#                     for idx in discrete_indices:
#                         if isinstance(idx, torch.Tensor):
#                             idx = idx.item()
#                         continuous_actions.append(discrete_actions[idx])
                    
#                     result.act = np.array(continuous_actions)
                
#                 return result
            
#             policy.forward = dqn_forward_with_continuous_output
#             optimizers.append(optim)
            
#         else:  # Random policy
#             policy = RandomPolicy(action_space)
#             optimizers.append(None)
        
#         policies.append(policy)
    
#     return policies, optimizers

# def train_agents(args):
#     """训练多智能体"""
#     print("初始化环境...")
    
#     # 环境设置
#     env = get_env(args)
#     train_envs = DummyVectorEnv([lambda: get_env(args) for _ in range(args.training_num)])
#     test_envs = DummyVectorEnv([lambda: get_env(args) for _ in range(args.test_num)])
    
#     # 设置随机种子
#     np.random.seed(args.seed)
#     torch.manual_seed(args.seed)
#     train_envs.seed(args.seed)
#     test_envs.seed(args.seed)
    
#     print("创建策略...")
#     # 创建智能体策略
#     policies, optimizers = create_simple_policies(args, env)
    
#     # 创建多智能体策略管理器
#     policy_manager = MultiAgentPolicyManager(policies, env)
    
#     print("创建收集器...")
#     # 创建收集器
#     train_collector = Collector(
#         policy_manager,
#         train_envs,
#         VectorReplayBuffer(args.buffer_size, len(train_envs)),
#         exploration_noise=True
#     )
    
#     test_collector = Collector(
#         policy_manager, 
#         test_envs,
#         exploration_noise=True
#     )
    
#     # 预收集一些数据
#     print("预收集数据...")
#     train_collector.collect(n_step=args.batch_size * args.training_num)
    
#     # 回调函数
#     def save_best_fn(policy):
#         """保存最佳策略"""
#         save_dir = "waterworld_policies"
#         os.makedirs(save_dir, exist_ok=True)
#         for i, agent_id in enumerate(env.agents):
#             if hasattr(policy.policies[agent_id], 'state_dict'):
#                 model_path = os.path.join(save_dir, f"{agent_id}_policy.pth")
#                 torch.save(policy.policies[agent_id].state_dict(), model_path)
#         print(f"策略已保存到 {save_dir}")
    
#     def stop_fn(mean_rewards):
#         """停止条件"""
#         return mean_rewards >= 5.0
    
#     def train_fn(epoch, env_step):
#         """训练时设置探索率"""
#         for agent_id in env.agents:
#             if hasattr(policy_manager.policies[agent_id], 'set_eps'):
#                 policy_manager.policies[agent_id].set_eps(0.1)
    
#     def test_fn(epoch, env_step):
#         """测试时设置探索率"""
#         for agent_id in env.agents:
#             if hasattr(policy_manager.policies[agent_id], 'set_eps'):
#                 policy_manager.policies[agent_id].set_eps(0.05)
    
#     def reward_metric(rews):
#         """奖励度量函数 - 返回所有agent的平均奖励"""
#         return rews.mean(axis=1)
    
#     # 检查是否有可训练的策略
#     trainable_policies = [p for p in policies if not isinstance(p, RandomPolicy)]
    
#     if trainable_policies:
#         print(f"开始训练 {len(trainable_policies)} 个可训练策略...")
        
#         # 使用off-policy训练器
#         result = offpolicy_trainer(
#             policy=policy_manager,
#             train_collector=train_collector,
#             test_collector=test_collector,
#             max_epoch=args.epoch,
#             step_per_epoch=args.step_per_epoch,
#             step_per_collect=args.step_per_collect,
#             episode_per_test=5,
#             batch_size=args.batch_size,
#             train_fn=train_fn,
#             test_fn=test_fn,
#             stop_fn=stop_fn,
#             save_best_fn=save_best_fn,
#             update_per_step=0.1,
#             test_in_train=False,
#             reward_metric=reward_metric
#         )
        
#         print("训练完成！")
#         print(f"训练结果: {result}")
#     else:
#         print("所有策略都是随机策略，运行测试...")
#         result = test_collector.collect(n_episode=5)
#         print(f"随机策略测试结果 - 平均奖励: {result['rew']}, 平均长度: {result['len']}")
    
#     return policy_manager, result

# def watch_performance(args):
#     """观察策略表现"""
#     print("观察策略表现...")
#     env = get_env(args)
#     policies, _ = create_simple_policies(args, env)
#     policy_manager = MultiAgentPolicyManager(policies, env)
    
#     # 尝试加载训练好的策略
#     save_dir = "waterworld_policies"
#     if os.path.exists(save_dir):
#         for i, agent_id in enumerate(env.agents):
#             if hasattr(policy_manager.policies[agent_id], 'load_state_dict'):
#                 model_path = os.path.join(save_dir, f"{agent_id}_policy.pth")
#                 if os.path.exists(model_path):
#                     policy_manager.policies[agent_id].load_state_dict(torch.load(model_path))
#                     print(f"加载了 {agent_id} 的策略")
    
#     # 需要使用向量化环境
#     test_envs = DummyVectorEnv([lambda: get_env(args) for _ in range(1)])
#     collector = Collector(policy_manager, test_envs)
    
#     print("运行测试episode...")
#     result = collector.collect(n_episode=3, render=args.render)
#     print(f"平均奖励: {result['rew']}")
#     print(f"平均长度: {result['len']}")

# def test_random_policies(args):
#     """测试纯随机策略作为基准"""
#     print("测试纯随机策略...")
#     env = get_env(args)
    
#     # 创建所有随机策略
#     policies = [RandomPolicy(env.action_space) for _ in env.agents]
#     policy_manager = MultiAgentPolicyManager(policies, env)
    
#     # 需要使用向量化环境
#     test_envs = DummyVectorEnv([lambda: get_env(args) for _ in range(1)])
    
#     # 运行测试
#     collector = Collector(policy_manager, test_envs)
#     result = collector.collect(n_episode=5, render=args.render)
#     print(f"随机策略基准 - 平均奖励: {result['rew']}, 平均长度: {result['len']}")
    
#     return result

# def simple_debug_test(args):
#     """最简单的debug测试"""
#     print("=== Debug测试：基础环境和策略创建 ===")
    
#     try:
#         # 1. 测试环境创建
#         print("1. 创建环境...")
#         env = get_env(args)
#         print(f"   环境创建成功: {type(env)}")
#         print(f"   智能体: {env.agents}")
#         print(f"   观察空间: {env.observation_space}")
#         print(f"   动作空间: {env.action_space}")
        
#         # 2. 测试策略创建
#         print("\n2. 创建策略...")
#         policies, optimizers = create_simple_policies(args, env)
#         print(f"   策略创建成功: {len(policies)} 个策略")
        
#         # 3. 测试策略管理器
#         print("\n3. 创建策略管理器...")
#         policy_manager = MultiAgentPolicyManager(policies, env)
#         print(f"   策略管理器创建成功: {type(policy_manager)}")
        
#         # 4. 测试向量化环境
#         print("\n4. 创建向量化环境...")
#         test_envs = DummyVectorEnv([lambda: get_env(args) for _ in range(1)])
#         print(f"   向量化环境创建成功: {type(test_envs)}")
        
#         # 5. 测试收集器
#         print("\n5. 创建收集器...")
#         collector = Collector(policy_manager, test_envs)
#         print(f"   收集器创建成功: {type(collector)}")
        
#         # 6. 测试收集一个step
#         print("\n6. 测试收集数据...")
#         result = collector.collect(n_step=10)
#         print(f"   数据收集成功: {result}")
        
#         print("\n=== 所有基础测试通过！===")
#         return True
        
#     except Exception as e:
#         print(f"\n!!! 测试失败: {e}")
#         import traceback
#         traceback.print_exc()
#         return False

# if __name__ == "__main__":
#     args = get_args()
    
#     print("=== Tianshou Waterworld 多智能体测试 ===")
#     print(f"设备: {args.device}")
#     print(f"智能体数量: {args.n_pursuers}")
    
#     # 首先运行debug测试
#     debug_success = simple_debug_test(args)
    
#     if not debug_success:
#         print("基础测试失败，请检查环境配置")
#         exit(1)
    
#     if args.watch:
#         watch_performance(args)
#     else:
#         # 首先测试随机策略
#         print("\n1. 测试随机策略基准:")
#         test_random_policies(args)
        
#         # 然后训练DQN策略
#         print("\n2. 训练DQN策略:")
#         policy_manager, result = train_agents(args)
        
#         # 训练完成后观察表现
#         if result:
#             print("\n3. 观察训练后的策略表现:")
#             args.render = True
#             watch_performance(args)

# """
# 使用说明：

# 1. 基础测试（只用随机策略）：
#    python waterworld_simple.py

# 2. 观察训练结果：
#    python waterworld_simple.py --watch --render

# 3. 自定义参数：
#    python waterworld_simple.py --epoch 20 --lr 1e-3

# 这个简化版本的特点：

# 1. 使用DQN处理连续动作空间（通过离散化）
# 2. 混合DQN和随机策略
# 3. 更强的错误处理和兼容性
# 4. 渐进式测试方法

# 如果这个版本能正常运行，我们再逐步添加更复杂的算法。
# """

In [9]:
# """
# Tianshou Waterworld 修复版本
# 解决RandomPolicy在连续动作空间的mask问题

# 每个agent使用独立的策略：
# - DQN (通过离散化处理连续动作)
# - 自定义连续随机策略
# """

# import os
# import argparse
# import numpy as np
# import torch
# from typing import Optional, Tuple, List

# # Tianshou imports
# from tianshou.data import Collector, VectorReplayBuffer, Batch
# from tianshou.env import DummyVectorEnv
# from tianshou.env.pettingzoo_env import PettingZooEnv
# from tianshou.policy import (
#     BasePolicy, 
#     DQNPolicy, 
#     MultiAgentPolicyManager
# )
# from tianshou.trainer import offpolicy_trainer
# from tianshou.utils.net.common import Net

# # PettingZoo imports
# from pettingzoo.sisl import waterworld_v4

# class ContinuousRandomPolicy(BasePolicy):
#     """自定义连续动作随机策略"""
#     def __init__(self, action_space):
#         super().__init__(action_space=action_space)
#         self.action_space = action_space
    
#     def forward(self, batch, state=None, **kwargs):
#         """前向传播，直接采样随机动作"""
#         if hasattr(batch.obs, 'shape') and len(batch.obs.shape) > 1:
#             batch_size = batch.obs.shape[0]
#         else:
#             batch_size = 1
            
#         actions = []
#         for _ in range(batch_size):
#             action = self.action_space.sample()
#             actions.append(action)
        
#         return Batch(act=np.array(actions))
    
#     def exploration_noise(self, act, batch):
#         """随机策略不需要额外的探索噪声"""
#         return act
    
#     def learn(self, batch, **kwargs):
#         """随机策略不需要学习"""
#         return {}

# def get_args():
#     """配置参数"""
#     parser = argparse.ArgumentParser()
#     parser.add_argument('--seed', type=int, default=42)
#     parser.add_argument('--buffer-size', type=int, default=20000)
#     parser.add_argument('--lr', type=float, default=1e-4)
#     parser.add_argument('--gamma', type=float, default=0.99)
#     parser.add_argument('--epoch', type=int, default=5)  # 进一步减少训练轮数
#     parser.add_argument('--step-per-epoch', type=int, default=500)
#     parser.add_argument('--step-per-collect', type=int, default=100)
#     parser.add_argument('--batch-size', type=int, default=64)
#     parser.add_argument('--hidden-sizes', type=int, nargs='*', default=[128, 128])
#     parser.add_argument('--training-num', type=int, default=2)  # 减少并行环境数
#     parser.add_argument('--test-num', type=int, default=1)
#     parser.add_argument('--device', type=str, default='cuda' if torch.cuda.is_available() else 'cpu')
#     parser.add_argument('--watch', action='store_true', default=False)
#     parser.add_argument('--render', action='store_true', default=False)
    
#     # Waterworld specific parameters
#     parser.add_argument('--n-pursuers', type=int, default=5)
    
#     return parser.parse_known_args()[0]

# def get_env(args):
#     """创建Waterworld环境"""
#     env = waterworld_v4.env(
#         n_pursuers=args.n_pursuers,
#         n_evaders=5,
#         n_poisons=10,
#         n_coop=2,
#         n_sensors=30,
#         sensor_range=0.2,
#         radius=0.015,
#         pursuer_max_accel=0.01,
#         evader_speed=0.01,
#         poison_speed=0.01,
#         poison_reward=-1.0,
#         food_reward=10.0,
#         encounter_reward=0.01,
#         thrust_penalty=-0.5,
#         local_ratio=1.0,
#         speed_features=True,
#         max_cycles=500,
#         render_mode="human" if args.render else None
#     )
#     return PettingZooEnv(env)

# def create_dqn_policy(args, state_shape, action_space, discrete_actions):
#     """创建DQN策略"""
#     # 创建DQN网络
#     net = Net(
#         state_shape=state_shape,
#         action_shape=len(discrete_actions),  # 9个离散动作
#         hidden_sizes=args.hidden_sizes,
#         device=args.device
#     ).to(args.device)
    
#     optim = torch.optim.Adam(net.parameters(), lr=args.lr)
    
#     # 创建DQN策略
#     policy = DQNPolicy(
#         model=net,
#         optim=optim,
#         discount_factor=args.gamma,
#         estimation_step=3,
#         target_update_freq=320
#     )
    
#     # 修复max_action_num属性
#     policy.max_action_num = len(discrete_actions)
    
#     # 添加动作转换函数
#     policy.discrete_actions = discrete_actions
#     original_forward = policy.forward
#     original_exploration_noise = policy.exploration_noise
    
#     def dqn_forward_with_continuous_output(batch, state=None, **kwargs):
#         # 调用原始forward得到离散动作
#         result = original_forward(batch, state, **kwargs)
        
#         # 将离散动作索引转换为连续动作
#         if hasattr(result, 'act'):
#             discrete_indices = result.act
#             continuous_actions = []
            
#             # 处理不同的batch结构
#             if isinstance(discrete_indices, np.ndarray):
#                 for idx in discrete_indices:
#                     if isinstance(idx, (torch.Tensor, np.ndarray)):
#                         idx = int(idx.item() if hasattr(idx, 'item') else idx)
#                     continuous_actions.append(discrete_actions[idx])
#             else:
#                 if isinstance(discrete_indices, (torch.Tensor, np.ndarray)):
#                     idx = int(discrete_indices.item() if hasattr(discrete_indices, 'item') else discrete_indices)
#                 else:
#                     idx = int(discrete_indices)
#                 continuous_actions.append(discrete_actions[idx])
            
#             result.act = np.array(continuous_actions)
        
#         return result
    
#     def custom_exploration_noise(act, batch):
#         """自定义探索噪声函数"""
#         if not hasattr(policy, 'eps'):
#             return act
            
#         bsz = len(act) if hasattr(act, '__len__') else 1
#         rand_mask = np.random.rand(bsz) < policy.eps
        
#         if np.any(rand_mask):
#             # 对需要随机的动作，直接从discrete_actions中随机选择
#             for i in range(bsz):
#                 if rand_mask[i]:
#                     random_action_idx = np.random.randint(0, len(discrete_actions))
#                     act[i] = discrete_actions[random_action_idx]
        
#         return act
    
#     policy.forward = dqn_forward_with_continuous_output
#     policy.exploration_noise = custom_exploration_noise
    
#     return policy, optim

# def create_policies(args, env) -> Tuple[List[BasePolicy], List]:
#     """创建策略组合"""
#     agents = env.agents
#     policies = []
#     optimizers = []
    
#     # 获取环境信息
#     observation_space = env.observation_space
#     action_space = env.action_space
    
#     state_shape = observation_space.shape
    
#     print(f"状态空间维度: {state_shape}")
#     print(f"动作空间: {action_space}")
#     print(f"智能体数量: {len(agents)}")
    
#     # 离散化动作
#     discrete_actions = [
#         [-0.01, -0.01], [-0.01, 0.0], [-0.01, 0.01],  # 左上, 左, 左下
#         [0.0, -0.01],   [0.0, 0.0],   [0.0, 0.01],    # 上, 停止, 下  
#         [0.01, -0.01],  [0.01, 0.0],  [0.01, 0.01]    # 右上, 右, 右下
#     ]
    
#     # 为每个agent分配策略
#     algorithm_assignment = {
#         'pursuer_0': 'DQN',
#         'pursuer_1': 'DQN',
#         'pursuer_2': 'Random',
#         'pursuer_3': 'Random',
#         'pursuer_4': 'DQN',
#     }
    
#     for agent_id in agents:
#         algo = algorithm_assignment.get(agent_id, 'Random')
#         print(f"Agent {agent_id} 使用算法: {algo}")
        
#         if algo == 'DQN':
#             policy, optim = create_dqn_policy(args, state_shape, action_space, discrete_actions)
#             optimizers.append(optim)
#         else:  # Random policy
#             policy = ContinuousRandomPolicy(action_space)
#             optimizers.append(None)
        
#         policies.append(policy)
    
#     return policies, optimizers

# def simple_test(args):
#     """简单测试"""
#     print("=== 简单测试 ===")
    
#     # 创建环境
#     env = get_env(args)
#     print(f"环境创建成功，智能体: {env.agents}")
    
#     # 创建策略
#     policies, optimizers = create_policies(args, env)
#     print(f"策略创建成功: {len(policies)} 个")
    
#     # 创建策略管理器
#     policy_manager = MultiAgentPolicyManager(policies, env)
#     print("策略管理器创建成功")
    
#     # 创建向量化环境
#     test_envs = DummyVectorEnv([lambda: get_env(args) for _ in range(1)])
#     print("向量化环境创建成功")
    
#     # 创建收集器
#     collector = Collector(policy_manager, test_envs)
#     print("收集器创建成功")
    
#     # 收集数据测试
#     print("开始收集测试数据...")
#     result = collector.collect(n_step=20)
#     print(f"测试成功！收集了 {result['n/st']} 步数据")
    
#     return True

# def train_agents(args):
#     """训练智能体"""
#     print("=== 开始训练 ===")
    
#     # 环境设置
#     env = get_env(args)
#     train_envs = DummyVectorEnv([lambda: get_env(args) for _ in range(args.training_num)])
#     test_envs = DummyVectorEnv([lambda: get_env(args) for _ in range(args.test_num)])
    
#     # 设置随机种子
#     np.random.seed(args.seed)
#     torch.manual_seed(args.seed)
#     train_envs.seed(args.seed)
#     test_envs.seed(args.seed)
    
#     # 创建策略
#     policies, optimizers = create_policies(args, env)
#     policy_manager = MultiAgentPolicyManager(policies, env)
    
#     # 创建收集器
#     train_collector = Collector(
#         policy_manager,
#         train_envs,
#         VectorReplayBuffer(args.buffer_size, len(train_envs)),
#         exploration_noise=True
#     )
    
#     test_collector = Collector(policy_manager, test_envs)
    
#     # 预收集数据
#     print("预收集训练数据...")
#     train_collector.collect(n_step=args.batch_size * args.training_num)
    
#     # 检查可训练策略
#     trainable_policies = [p for p in policies if hasattr(p, 'optim')]
#     print(f"发现 {len(trainable_policies)} 个可训练策略")
    
#     if trainable_policies:
#         print("开始训练...")
        
#         # 回调函数
#         def save_best_fn(policy):
#             save_dir = "waterworld_policies"
#             os.makedirs(save_dir, exist_ok=True)
#             print(f"策略已保存到 {save_dir}")
        
#         def stop_fn(mean_rewards):
#             return mean_rewards >= 3.0  # 降低停止条件
        
#         def train_fn(epoch, env_step):
#             for agent_id in env.agents:
#                 if hasattr(policy_manager.policies[agent_id], 'set_eps'):
#                     policy_manager.policies[agent_id].set_eps(0.1)
        
#         def test_fn(epoch, env_step):
#             for agent_id in env.agents:
#                 if hasattr(policy_manager.policies[agent_id], 'set_eps'):
#                     policy_manager.policies[agent_id].set_eps(0.05)
        
#         def reward_metric(rews):
#             return rews.mean(axis=1)
        
#         # 开始训练
#         result = offpolicy_trainer(
#             policy=policy_manager,
#             train_collector=train_collector,
#             test_collector=test_collector,
#             max_epoch=args.epoch,
#             step_per_epoch=args.step_per_epoch,
#             step_per_collect=args.step_per_collect,
#             episode_per_test=3,
#             batch_size=args.batch_size,
#             train_fn=train_fn,
#             test_fn=test_fn,
#             stop_fn=stop_fn,
#             save_best_fn=save_best_fn,
#             update_per_step=0.1,
#             test_in_train=False,
#             reward_metric=reward_metric
#         )
        
#         print("训练完成！")
#         print(f"最终结果: {result}")
#         return policy_manager, result
#     else:
#         print("没有可训练策略，运行随机测试...")
#         result = test_collector.collect(n_episode=3)
#         print(f"随机测试结果: {result}")
#         return policy_manager, result

# def watch_performance(args):
#     """观察性能"""
#     print("=== 观察性能 ===")
    
#     env = get_env(args)
#     policies, _ = create_policies(args, env)
#     policy_manager = MultiAgentPolicyManager(policies, env)
    
#     test_envs = DummyVectorEnv([lambda: get_env(args) for _ in range(1)])
#     collector = Collector(policy_manager, test_envs)
    
#     result = collector.collect(n_episode=3, render=args.render)
#     print(f"性能测试结果:")
#     print(f"  平均奖励: {result['rew']}")
#     print(f"  平均长度: {result['len']}")

# if __name__ == "__main__":
#     args = get_args()
    
#     print("=== Tianshou Waterworld 修复版本 ===")
#     print(f"设备: {args.device}")
#     print(f"智能体数量: {args.n_pursuers}")
    
#     # 1. 简单测试
#     print("\n1. 运行简单测试...")
#     if not simple_test(args):
#         print("简单测试失败")
#         exit(1)
    
#     if args.watch:
#         # 2. 观察性能
#         watch_performance(args)
#     else:
#         # 3. 训练
#         print("\n2. 开始训练...")
#         policy_manager, result = train_agents(args)
        
#         # 4. 观察结果
#         print("\n3. 观察训练结果...")
#         watch_performance(args)

# """
# 使用说明：

# 1. 基础运行:
#    python waterworld_fixed.py

# 2. 观察模式:
#    python waterworld_fixed.py --watch --render

# 3. 快速测试:
#    python waterworld_fixed.py --epoch 2 --step-per-epoch 200

# 修复内容:
# - 自定义ContinuousRandomPolicy解决mask问题
# - 简化DQN动作转换逻辑
# - 更好的错误处理
# - 渐进式测试流程
# """

In [10]:
# """
# Tianshou Waterworld 稳定版本
# 先用纯随机策略验证框架，然后逐步添加可训练策略

# 每个agent使用不同类型的随机策略来模拟"独立算法"的概念
# """

# import os
# import argparse
# import numpy as np
# import torch
# from typing import Optional, Tuple, List

# # Tianshou imports
# from tianshou.data import Collector, Batch
# from tianshou.env import DummyVectorEnv
# from tianshou.env.pettingzoo_env import PettingZooEnv
# from tianshou.policy import BasePolicy, MultiAgentPolicyManager

# # PettingZoo imports
# from pettingzoo.sisl import waterworld_v4

# class ContinuousRandomPolicy(BasePolicy):
#     """基础连续随机策略"""
#     def __init__(self, action_space, policy_name="Random"):
#         super().__init__(action_space=action_space)
#         self.action_space = action_space
#         self.policy_name = policy_name
    
#     def forward(self, batch, state=None, **kwargs):
#         if hasattr(batch.obs, 'shape') and len(batch.obs.shape) > 1:
#             batch_size = batch.obs.shape[0]
#         else:
#             batch_size = 1
            
#         actions = []
#         for _ in range(batch_size):
#             action = self.action_space.sample()
#             actions.append(action)
        
#         return Batch(act=np.array(actions))
    
#     def exploration_noise(self, act, batch):
#         return act
    
#     def learn(self, batch, **kwargs):
#         return {}

# class AggressiveRandomPolicy(ContinuousRandomPolicy):
#     """激进随机策略 - 偏向更大的动作"""
#     def __init__(self, action_space):
#         super().__init__(action_space, "Aggressive")
    
#     def forward(self, batch, state=None, **kwargs):
#         if hasattr(batch.obs, 'shape') and len(batch.obs.shape) > 1:
#             batch_size = batch.obs.shape[0]
#         else:
#             batch_size = 1
            
#         actions = []
#         for _ in range(batch_size):
#             # 生成偏向边界的动作
#             action = np.random.uniform(-1.0, 1.0, size=self.action_space.shape)
#             # 增强动作幅度
#             action = np.sign(action) * np.abs(action) * 0.8  # 偏向较大动作
#             # 确保在范围内
#             action = np.clip(action, self.action_space.low, self.action_space.high)
#             actions.append(action)
        
#         return Batch(act=np.array(actions))

# class ConservativeRandomPolicy(ContinuousRandomPolicy):
#     """保守随机策略 - 偏向较小的动作"""
#     def __init__(self, action_space):
#         super().__init__(action_space, "Conservative")
    
#     def forward(self, batch, state=None, **kwargs):
#         if hasattr(batch.obs, 'shape') and len(batch.obs.shape) > 1:
#             batch_size = batch.obs.shape[0]
#         else:
#             batch_size = 1
            
#         actions = []
#         for _ in range(batch_size):
#             # 生成偏向中心的动作
#             action = np.random.uniform(-0.3, 0.3, size=self.action_space.shape)  # 较小范围
#             # 确保在范围内
#             action = np.clip(action, self.action_space.low, self.action_space.high)
#             actions.append(action)
        
#         return Batch(act=np.array(actions))

# class BiasedRandomPolicy(ContinuousRandomPolicy):
#     """偏向随机策略 - 偏向某个方向"""
#     def __init__(self, action_space, bias_direction=None):
#         super().__init__(action_space, "Biased")
#         # 如果没有指定偏向，随机选择一个
#         self.bias = bias_direction if bias_direction is not None else np.random.uniform(-0.5, 0.5, size=action_space.shape)
    
#     def forward(self, batch, state=None, **kwargs):
#         if hasattr(batch.obs, 'shape') and len(batch.obs.shape) > 1:
#             batch_size = batch.obs.shape[0]
#         else:
#             batch_size = 1
            
#         actions = []
#         for _ in range(batch_size):
#             # 基础随机动作
#             base_action = np.random.uniform(-0.5, 0.5, size=self.action_space.shape)
#             # 添加偏向
#             action = base_action + self.bias
#             # 确保在范围内
#             action = np.clip(action, self.action_space.low, self.action_space.high)
#             actions.append(action)
        
#         return Batch(act=np.array(actions))

# def get_args():
#     """配置参数"""
#     parser = argparse.ArgumentParser()
#     parser.add_argument('--seed', type=int, default=42)
#     parser.add_argument('--n-episodes', type=int, default=10)
#     parser.add_argument('--device', type=str, default='cuda' if torch.cuda.is_available() else 'cpu')
#     parser.add_argument('--render', action='store_true', default=False)
#     parser.add_argument('--n-pursuers', type=int, default=5)
#     parser.add_argument('--env-num', type=int, default=1)
    
#     return parser.parse_known_args()[0]

# def get_env(args):
#     """创建Waterworld环境"""
#     env = waterworld_v4.env(
#         n_pursuers=args.n_pursuers,
#         n_evaders=5,
#         n_poisons=10,
#         n_coop=2,
#         n_sensors=30,
#         sensor_range=0.2,
#         radius=0.015,
#         pursuer_max_accel=0.01,
#         evader_speed=0.01,
#         poison_speed=0.01,
#         poison_reward=-1.0,
#         food_reward=10.0,
#         encounter_reward=0.01,
#         thrust_penalty=-0.5,
#         local_ratio=1.0,
#         speed_features=True,
#         max_cycles=500,
#         render_mode="human" if args.render else None
#     )
#     return PettingZooEnv(env)

# def create_diverse_policies(args, env):
#     """为每个agent创建不同类型的策略"""
#     agents = env.agents
#     policies = []
    
#     action_space = env.action_space
    
#     print(f"环境信息:")
#     print(f"  智能体: {agents}")
#     print(f"  观察空间: {env.observation_space}")
#     print(f"  动作空间: {action_space}")
    
#     # 为每个agent分配不同的策略类型
#     policy_types = [
#         ("Standard Random", ContinuousRandomPolicy),
#         ("Aggressive", AggressiveRandomPolicy), 
#         ("Conservative", ConservativeRandomPolicy),
#         ("Biased", BiasedRandomPolicy),
#         ("Standard Random 2", ContinuousRandomPolicy)
#     ]
    
#     for i, agent_id in enumerate(agents):
#         policy_name, policy_class = policy_types[i % len(policy_types)]
        
#         if policy_class == BiasedRandomPolicy:
#             # 为偏向策略设置随机偏向方向
#             bias = np.random.uniform(-0.3, 0.3, size=action_space.shape)
#             policy = policy_class(action_space, bias_direction=bias)
#         else:
#             policy = policy_class(action_space)
        
#         policies.append(policy)
#         print(f"  Agent {agent_id}: {policy_name}")
    
#     return policies

# def run_multi_agent_test(args):
#     """运行多智能体测试"""
#     print("=== 多智能体独立策略测试 ===")
    
#     # 创建环境
#     env = get_env(args)
    
#     # 创建不同的策略
#     policies = create_diverse_policies(args, env)
    
#     # 创建策略管理器
#     policy_manager = MultiAgentPolicyManager(policies, env)
#     print(f"\n策略管理器创建成功，管理 {len(policies)} 个独立策略")
    
#     # 创建向量化环境
#     vec_envs = DummyVectorEnv([lambda: get_env(args) for _ in range(args.env_num)])
    
#     # 创建收集器
#     collector = Collector(policy_manager, vec_envs)
#     print("收集器创建成功")
    
#     # 设置随机种子
#     np.random.seed(args.seed)
#     vec_envs.seed(args.seed)
    
#     # 运行测试
#     print(f"\n开始运行 {args.n_episodes} 个episode...")
#     result = collector.collect(n_episode=args.n_episodes, render=args.render)
    
#     # 分析结果
#     print(f"\n=== 测试结果 ===")
#     print(f"总episode数: {result['n/ep']}")
#     print(f"总步数: {result['n/st']}")
#     print(f"平均episode长度: {result['len']:.2f}")
#     print(f"平均奖励: {result['rew']:.4f}")
#     print(f"奖励标准差: {result['rew_std']:.4f}")
    
#     # 详细奖励分析
#     if 'rews' in result:
#         rewards = result['rews']
#         print(f"\n=== 详细奖励分析 ===")
#         print(f"奖励形状: {rewards.shape}")
#         print(f"最大奖励: {np.max(rewards):.4f}")
#         print(f"最小奖励: {np.min(rewards):.4f}")
        
#         # 如果是多智能体，分析每个agent的表现
#         if len(rewards.shape) > 1 and rewards.shape[1] > 1:
#             print(f"\n各智能体平均奖励:")
#             for i in range(rewards.shape[1]):
#                 agent_reward = np.mean(rewards[:, i])
#                 print(f"  Agent {i}: {agent_reward:.4f}")
    
#     return result

# def compare_policy_performance(args):
#     """比较不同策略的性能"""
#     print("\n=== 策略性能比较 ===")
    
#     env = get_env(args)
#     action_space = env.action_space
    
#     # 测试不同策略类型
#     policy_configs = [
#         ("Standard Random", ContinuousRandomPolicy, {}),
#         ("Aggressive", AggressiveRandomPolicy, {}),
#         ("Conservative", ConservativeRandomPolicy, {}),
#         ("Biased Forward", BiasedRandomPolicy, {"bias_direction": np.array([0.3, 0.0])}),
#         ("Biased Backward", BiasedRandomPolicy, {"bias_direction": np.array([-0.3, 0.0])})
#     ]
    
#     results = {}
    
#     for policy_name, policy_class, kwargs in policy_configs:
#         print(f"\n测试策略: {policy_name}")
        
#         # 创建该策略的所有agents
#         policies = [policy_class(action_space, **kwargs) for _ in env.agents]
#         policy_manager = MultiAgentPolicyManager(policies, env)
        
#         # 创建环境和收集器
#         vec_envs = DummyVectorEnv([lambda: get_env(args) for _ in range(1)])
#         collector = Collector(policy_manager, vec_envs)
        
#         # 运行测试
#         result = collector.collect(n_episode=5, render=False)
#         results[policy_name] = result['rew']
        
#         print(f"  平均奖励: {result['rew']:.4f}")
#         print(f"  平均长度: {result['len']:.2f}")
    
#     # 总结比较
#     print(f"\n=== 策略性能排序 ===")
#     sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)
#     for i, (policy_name, reward) in enumerate(sorted_results, 1):
#         print(f"{i}. {policy_name}: {reward:.4f}")

# def single_agent_test(args):
#     """单智能体测试验证基础功能"""
#     print("=== 单智能体基础测试 ===")
    
#     # 创建单智能体环境用于测试
#     args_single = argparse.Namespace(**vars(args))
#     args_single.n_pursuers = 1
    
#     env = get_env(args_single)
#     print(f"单智能体环境创建成功")
#     print(f"  智能体: {env.agents}")
    
#     # 创建单个策略
#     policy = ContinuousRandomPolicy(env.action_space)
#     policy_manager = MultiAgentPolicyManager([policy], env)
    
#     # 测试
#     vec_envs = DummyVectorEnv([lambda: get_env(args_single) for _ in range(1)])
#     collector = Collector(policy_manager, vec_envs)
    
#     result = collector.collect(n_episode=3, render=args.render)
#     print(f"单智能体测试成功！")
#     print(f"  平均奖励: {result['rew']:.4f}")
#     print(f"  平均长度: {result['len']:.2f}")
    
#     return True

# if __name__ == "__main__":
#     args = get_args()
    
#     print("=== Tianshou Waterworld 稳定版本 ===")
#     print(f"设备: {args.device}")
#     print(f"智能体数量: {args.n_pursuers}")
#     print(f"测试episode数: {args.n_episodes}")
    
#     try:
#         # 1. 单智能体基础测试
#         print("\n" + "="*50)
#         print("步骤1: 单智能体基础测试")
#         if not single_agent_test(args):
#             print("单智能体测试失败")
#             exit(1)
#         print("✅ 单智能体测试通过")
        
#         # 2. 多智能体测试
#         print("\n" + "="*50)
#         print("步骤2: 多智能体独立策略测试")
#         result = run_multi_agent_test(args)
#         print("✅ 多智能体测试通过")
        
#         # 3. 策略比较
#         if not args.render:  # 只在非渲染模式下进行比较测试
#             print("\n" + "="*50)
#             print("步骤3: 不同策略性能比较")
#             compare_policy_performance(args)
#             print("✅ 策略比较完成")
        
#         print(f"\n" + "="*60)
#         print(f"🎉 所有测试完成！")
#         print(f"")
#         print(f"📋 测试总结:")
#         print(f"   ✅ 环境创建和包装: PettingZoo → Tianshou")
#         print(f"   ✅ 多策略管理: MultiAgentPolicyManager")
#         print(f"   ✅ 独立策略: 每个agent使用不同策略")
#         print(f"   ✅ 数据收集: Collector正常工作")
#         print(f"   ✅ 连续动作空间: 正确处理")
#         print(f"")
#         print(f"🚀 下一步建议:")
#         print(f"   1. 这个框架已验证可以支持每个agent的独立策略")
#         print(f"   2. 可以逐个将随机策略替换为可训练算法")
#         print(f"   3. 建议顺序: 先试PPO(连续动作), 再试DQN(需要动作离散化)")
#         print(f"   4. 或者使用SAC等直接支持连续动作的算法")
#         print(f"")
#         print(f"💡 核心价值:")
#         print(f"   - 展示了Tianshou中真正的'每个agent独立算法'实现")
#         print(f"   - 为多智能体强化学习研究提供了稳定的基础框架")
#         print(f"   - 可以轻松扩展到任何PettingZoo环境")
        
#     except Exception as e:
#         print(f"\n❌ 测试过程中出现错误: {e}")
#         import traceback
#         traceback.print_exc()
#         print(f"\n💡 调试建议:")
#         print(f"   1. 检查依赖版本: pip list | grep -E '(tianshou|pettingzoo)'")
#         print(f"   2. 尝试简化参数: --n-episodes 3 --n-pursuers 3")
#         print(f"   3. 如有问题可以逐步调试每个组件")

# """
# 使用说明:

# 1. 基础测试:
#    python waterworld_stable.py

# 2. 可视化测试:
#    python waterworld_stable.py --render

# 3. 更多episode:
#    python waterworld_stable.py --n-episodes 20

# 4. 不同智能体数量:
#    python waterworld_stable.py --n-pursuers 3

# 这个版本的特点:
# - 完全避免了DQN的兼容性问题
# - 展示了真正的"每个agent独立策略"概念
# - 4种不同类型的策略模拟不同算法
# - 稳定的错误处理和测试流程
# - 性能比较和分析功能

# 这为后续添加真正的可训练算法（DQN、PPO、SAC等）奠定了坚实基础。
# """

In [11]:
# from tianshou.data import Collector
# from tianshou.env import DummyVectorEnv, PettingZooEnv
# from tianshou.policy import MultiAgentPolicyManager, RandomPolicy
# from pettingzoo.classic import leduc_holdem_v4
# from pettingzoo.classic import rps_v2
# from pettingzoo.sisl import pursuit_v4
# if __name__ == "__main__":
#     # Step 1: Load the PettingZoo environment
#     env = pursuit_v4.env(n_pursuers=2,render_mode="human")

#     # Step 2: Wrap the environment for Tianshou interfacing
#     env = PettingZooEnv(env)

#     # Step 3: Define policies for each agent
#     policies = MultiAgentPolicyManager([RandomPolicy(), RandomPolicy()], env)

#     # Step 4: Convert the env to vector format
#     env = DummyVectorEnv([lambda: env])

#     # Step 5: Construct the Collector, which interfaces the policies with the vectorised environment
#     collector = Collector(policies, env)

#     # Step 6: Execute the environment with the agents playing for 1 episode, and render a frame every 0.1 seconds
#     result = collector.collect(n_episode=1, render=0.1)

In [12]:
# from gymnasium import spaces

In [13]:
# # 如果你装的是 gymnasium，就用下面这行；否则改为 from gym import spaces
# from gymnasium import spaces  
# import numpy as np
# from pettingzoo.utils.wrappers.base import BaseWrapper

# class DiscreteActionWrapper(BaseWrapper):
#     """
#     将连续的 Box 动作空间离散化成 Discrete(n_bins^dim)。
#     """
#     def __init__(self, env, n_bins):
#         super().__init__(env)
#         self.n_bins = n_bins

#         # —— 解包，只解到 raw_env（有 possible_agents）为止 —— 
#         unwrapped = env
#         # 只要下一层有 env 且当前层没有 possible_agents，就往下一层
#         while hasattr(unwrapped, 'env') and not hasattr(unwrapped, 'possible_agents'):
#             unwrapped = unwrapped.env
#         raw_env = unwrapped  # 这时 raw_env 就是那个继承自 AECEnv 的实例

#         # 取第一个 agent，读它的 Box 下界和上界
#         sample_agent = raw_env.possible_agents[0]
#         box: spaces.Box = raw_env.action_space(sample_agent)
#         low, high = box.low, box.high
#         dims = low.shape[0]

#         # 为每个维度做等间隔分箱
#         grids = [np.linspace(low[i], high[i], n_bins) for i in range(dims)]
#         mesh  = np.meshgrid(*grids)
#         # 将网格展平成 (n_bins^dims, dims)  
#         self._actions = np.stack([m.flatten() for m in mesh], axis=1)

#         # 把所有 agent 的 action_space 都换成 Discrete
#         discrete_space = spaces.Discrete(self._actions.shape[0])
#         for agent in raw_env.possible_agents:
#             self.action_spaces[agent] = discrete_space

#     def step(self, action):
#         # 离散编号 → 连续向量 → 传给底层 env
#         cont = self._actions[action]
#         return self.env.step(cont)

#     def reset(self, seed=None, options=None):
#         return self.env.reset(seed=seed, options=options)


In [14]:
# """
# Multi-Agent Framework for PettingZoo Waterworld - 修复版本
# 支持每个智能体使用不同算法的框架

# 修复了常见的错误和兼容性问题
# """

# import numpy as np
# import gymnasium
# from typing import Dict, Any, Optional, Union, List
# from abc import ABC, abstractmethod
# import time
# import json
# import os
# from dataclasses import dataclass, asdict
# from pettingzoo.sisl import waterworld_v4
# import supersuit as ss


# @dataclass
# class AgentConfig:
#     """智能体配置类"""
#     agent_id: str
#     algorithm: str
#     hyperparameters: Dict[str, Any]
#     device: str = "cpu"
    
#     def to_dict(self):
#         return asdict(self)


# @dataclass
# class EnvironmentConfig:
#     """环境配置类"""
#     n_pursuers: int = 3
#     n_evaders: int = 3
#     n_poisons: int = 10
#     n_coop: int = 2
#     n_sensors: int = 20
#     sensor_range: float = 0.2
#     radius: float = 0.015
#     max_cycles: int = 500
#     render_mode: Optional[str] = None
    
#     def to_dict(self):
#         # 过滤掉None值，避免传递给环境时出错
#         config_dict = asdict(self)
#         return {k: v for k, v in config_dict.items() if v is not None}


# class BaseAgent(ABC):
#     """智能体基类 - 所有算法都需要继承这个类"""
    
#     def __init__(self, agent_id: str, observation_space: gymnasium.Space, 
#                  action_space: gymnasium.Space, config: Dict[str, Any]):
#         self.agent_id = agent_id
#         self.observation_space = observation_space
#         self.action_space = action_space
#         self.config = config
#         self.step_count = 0
#         self.total_reward = 0.0
        
#     @abstractmethod
#     def select_action(self, observation: np.ndarray, training: bool = True) -> np.ndarray:
#         """选择动作 - 子类必须实现"""
#         pass
    
#     @abstractmethod
#     def update(self, experience: Dict[str, Any]) -> Dict[str, float]:
#         """更新策略 - 子类必须实现"""
#         pass
    
#     @abstractmethod
#     def save_model(self, filepath: str) -> None:
#         """保存模型 - 子类必须实现"""
#         pass
    
#     @abstractmethod
#     def load_model(self, filepath: str) -> None:
#         """加载模型 - 子类必须实现"""
#         pass
    
#     def reset_episode_stats(self):
#         """重置回合统计"""
#         self.step_count = 0
#         self.total_reward = 0.0
    
#     def get_stats(self) -> Dict[str, float]:
#         """获取统计信息"""
#         return {
#             "steps": self.step_count,
#             "total_reward": self.total_reward,
#             "avg_reward": self.total_reward / max(1, self.step_count)
#         }


# class RandomAgent(BaseAgent):
#     """随机智能体 - 完全随机选择动作"""
    
#     def __init__(self, agent_id: str, observation_space: gymnasium.Space, 
#                  action_space: gymnasium.Space, config: Dict[str, Any]):
#         super().__init__(agent_id, observation_space, action_space, config)
#         self.algorithm_name = "Random"
        
#         # 创建独立的随机数生成器，避免全局影响
#         self.random_seed = config.get("random_seed", None)
#         self.rng = np.random.RandomState(self.random_seed)
    
#     def select_action(self, observation: np.ndarray, training: bool = True) -> np.ndarray:
#         """随机选择动作"""
#         # 手动生成随机动作，确保类型正确
#         if isinstance(self.action_space, gymnasium.spaces.Box):
#             action = self.rng.uniform(
#                 low=self.action_space.low,
#                 high=self.action_space.high,
#                 size=self.action_space.shape
#             ).astype(np.float32)
#         else:
#             action = self.action_space.sample()
#         return action
    
#     def update(self, experience: Dict[str, Any]) -> Dict[str, float]:
#         """随机智能体不需要更新"""
#         return {"loss": 0.0}
    
#     def save_model(self, filepath: str) -> None:
#         """随机智能体没有模型需要保存"""
#         config_data = {
#             "agent_id": self.agent_id,
#             "algorithm": self.algorithm_name,
#             "config": self.config
#         }
#         os.makedirs(os.path.dirname(filepath), exist_ok=True)
#         with open(filepath, 'w') as f:
#             json.dump(config_data, f, indent=2)
    
#     def load_model(self, filepath: str) -> None:
#         """随机智能体没有模型需要加载"""
#         pass


# class ConstrainedRandomAgent(BaseAgent):
#     """受约束的随机智能体 - 在一定范围内随机选择动作"""
    
#     def __init__(self, agent_id: str, observation_space: gymnasium.Space, 
#                  action_space: gymnasium.Space, config: Dict[str, Any]):
#         super().__init__(agent_id, observation_space, action_space, config)
#         self.algorithm_name = "ConstrainedRandom"
        
#         # 约束参数
#         self.action_scale = config.get("action_scale", 1.0)
#         bias_config = config.get("bias", [0.0] * action_space.shape[0])
#         self.bias = np.array(bias_config, dtype=np.float32)
        
#         # 创建独立的随机数生成器
#         self.random_seed = config.get("random_seed", None)
#         self.rng = np.random.RandomState(self.random_seed)
    
#     def select_action(self, observation: np.ndarray, training: bool = True) -> np.ndarray:
#         """在约束范围内随机选择动作"""
#         # 生成基础随机动作
#         if isinstance(self.action_space, gymnasium.spaces.Box):
#             raw_action = self.rng.uniform(
#                 low=self.action_space.low,
#                 high=self.action_space.high,
#                 size=self.action_space.shape
#             ).astype(np.float32)
#         else:
#             raw_action = self.action_space.sample()
        
#         # 应用缩放和偏置
#         action = raw_action * self.action_scale + self.bias
        
#         # 确保动作在有效范围内
#         if isinstance(self.action_space, gymnasium.spaces.Box):
#             action = np.clip(action, self.action_space.low, self.action_space.high)
        
#         return action
    
#     def update(self, experience: Dict[str, Any]) -> Dict[str, float]:
#         """约束随机智能体不需要更新"""
#         return {"loss": 0.0}
    
#     def save_model(self, filepath: str) -> None:
#         """保存配置信息"""
#         config_data = {
#             "agent_id": self.agent_id,
#             "algorithm": self.algorithm_name,
#             "config": self.config,
#             "action_scale": self.action_scale,
#             "bias": self.bias.tolist()
#         }
#         os.makedirs(os.path.dirname(filepath), exist_ok=True)
#         with open(filepath, 'w') as f:
#             json.dump(config_data, f, indent=2)
    
#     def load_model(self, filepath: str) -> None:
#         """加载配置信息"""
#         with open(filepath, 'r') as f:
#             data = json.load(f)
#             self.action_scale = data.get("action_scale", 1.0)
#             self.bias = np.array(data.get("bias", [0.0] * self.action_space.shape[0]), dtype=np.float32)


# class AgentFactory:
#     """智能体工厂类 - 负责创建不同类型的智能体"""
    
#     _agent_classes = {
#         "random": RandomAgent,
#         "constrained_random": ConstrainedRandomAgent,
#     }
    
#     @classmethod
#     def create_agent(cls, agent_id: str, algorithm: str, 
#                     observation_space: gymnasium.Space, action_space: gymnasium.Space,
#                     config: Dict[str, Any]) -> BaseAgent:
#         """创建智能体实例"""
#         if algorithm.lower() not in cls._agent_classes:
#             raise ValueError(f"Unknown algorithm: {algorithm}. Available: {list(cls._agent_classes.keys())}")
        
#         agent_class = cls._agent_classes[algorithm.lower()]
#         return agent_class(agent_id, observation_space, action_space, config)
    
#     @classmethod
#     def register_agent(cls, algorithm_name: str, agent_class: type):
#         """注册新的智能体类型"""
#         cls._agent_classes[algorithm_name.lower()] = agent_class
    
#     @classmethod
#     def get_available_algorithms(cls) -> List[str]:
#         """获取所有可用算法"""
#         return list(cls._agent_classes.keys())


# class MultiAgentEnvironment:
#     """多智能体环境管理器"""
    
#     def __init__(self, env_config: EnvironmentConfig):
#         self.env_config = env_config
#         self.env = None
#         self.agents = {}
#         self.agent_configs = {}
#         self.episode_stats = {}
        
#     def create_environment(self):
#         """创建环境"""
#         try:
#             # 创建原始环境
#             env_params = self.env_config.to_dict()
#             print(f"创建环境参数: {env_params}")
            
#             self.env = waterworld_v4.env(**env_params)
            
#             # 应用预处理
#             self.env = ss.flatten_v0(self.env)  # 扁平化观察空间
            
#             # 重置环境以初始化智能体
#             self.env.reset()
            
#             print(f"环境创建成功，智能体: {self.env.possible_agents}")
            
#             # 验证环境
#             if not self.env.possible_agents:
#                 raise RuntimeError("环境中没有智能体")
                
#             return self.env
            
#         except Exception as e:
#             print(f"创建环境失败: {str(e)}")
#             raise
    
#     def add_agent(self, agent_config: AgentConfig):
#         """添加智能体"""
#         if self.env is None:
#             raise RuntimeError("Environment not created. Call create_environment() first.")
        
#         agent_id = agent_config.agent_id
        
#         # 检查智能体是否存在于环境中
#         if agent_id not in self.env.possible_agents:
#             raise ValueError(f"Agent {agent_id} not found in environment. Available: {self.env.possible_agents}")
        
#         try:
#             # 获取智能体的观察空间和动作空间
#             obs_space = self.env.observation_space(agent_id)
#             action_space = self.env.action_space(agent_id)
            
#             print(f"智能体 {agent_id}: 观察空间={obs_space}, 动作空间={action_space}")
            
#             # 创建智能体实例
#             agent = AgentFactory.create_agent(
#                 agent_id=agent_id,
#                 algorithm=agent_config.algorithm,
#                 observation_space=obs_space,
#                 action_space=action_space,
#                 config=agent_config.hyperparameters
#             )
            
#             self.agents[agent_id] = agent
#             self.agent_configs[agent_id] = agent_config
            
#             print(f"智能体 {agent_id} 添加成功，算法: {agent_config.algorithm}")
            
#         except Exception as e:
#             print(f"添加智能体 {agent_id} 失败: {str(e)}")
#             raise
    
#     def run_episode(self, max_steps: Optional[int] = None, render: bool = False) -> Dict[str, Any]:
#         """运行一个回合"""
#         if self.env is None or not self.agents:
#             raise RuntimeError("Environment and agents must be set up first.")
        
#         try:
#             # 重置环境和智能体统计
#             self.env.reset()
#             for agent in self.agents.values():
#                 agent.reset_episode_stats()
            
#             episode_rewards = {agent_id: 0.0 for agent_id in self.agents.keys()}
#             episode_steps = 0
#             episode_start_time = time.time()
            
#             # 存储经验的缓冲区
#             experiences = {agent_id: [] for agent_id in self.agents.keys()}
            
#             # 运行回合
#             while self.env.agents and (max_steps is None or episode_steps < max_steps):
#                 try:
#                     # 获取当前智能体
#                     current_agent_id = self.env.agent_selection
                    
#                     # 获取观察
#                     observation, reward, termination, truncation, info = self.env.last()
                    
#                     # 更新奖励统计
#                     if current_agent_id in episode_rewards:
#                         episode_rewards[current_agent_id] += reward
#                         if current_agent_id in self.agents:
#                             self.agents[current_agent_id].total_reward += reward
#                             self.agents[current_agent_id].step_count += 1
                    
#                     # 如果智能体结束，执行 None 动作
#                     if termination or truncation:
#                         action = None
#                     else:
#                         # 选择动作
#                         if current_agent_id in self.agents:
#                             action = self.agents[current_agent_id].select_action(observation)
#                         else:
#                             # 如果没有对应的智能体，随机选择动作
#                             action = self.env.action_space(current_agent_id).sample()
                    
#                     # 存储经验（用于未来的学习算法）
#                     if current_agent_id in experiences and not (termination or truncation):
#                         experience = {
#                             "observation": observation.copy() if hasattr(observation, 'copy') else observation,
#                             "action": action.copy() if hasattr(action, 'copy') else action,
#                             "reward": reward,
#                             "done": termination or truncation,
#                             "info": info
#                         }
#                         experiences[current_agent_id].append(experience)
                    
#                     # 执行动作
#                     self.env.step(action)
#                     episode_steps += 1
                    
#                     # 渲染
#                     if render:
#                         self.env.render()
                        
#                 except Exception as e:
#                     print(f"回合执行中出错 (步骤 {episode_steps}): {str(e)}")
#                     break
            
#             episode_duration = time.time() - episode_start_time
            
#             # 收集回合统计信息
#             episode_stats = {
#                 "episode_steps": episode_steps,
#                 "episode_duration": episode_duration,
#                 "episode_rewards": episode_rewards,
#                 "total_reward": sum(episode_rewards.values()),
#                 "avg_reward": sum(episode_rewards.values()) / max(1, len(episode_rewards)),
#                 "agent_stats": {agent_id: agent.get_stats() for agent_id, agent in self.agents.items()}
#             }
            
#             return episode_stats
            
#         except Exception as e:
#             print(f"运行回合时出错: {str(e)}")
#             raise
    
#     def run_multiple_episodes(self, num_episodes: int, max_steps_per_episode: Optional[int] = None,
#                             render: bool = False, verbose: bool = True) -> List[Dict[str, Any]]:
#         """运行多个回合"""
#         all_episode_stats = []
        
#         for episode in range(num_episodes):
#             try:
#                 if verbose:
#                     print(f"\n--- 回合 {episode + 1}/{num_episodes} ---")
                
#                 episode_stats = self.run_episode(max_steps_per_episode, render)
#                 all_episode_stats.append(episode_stats)
                
#                 if verbose:
#                     print(f"回合步数: {episode_stats['episode_steps']}")
#                     print(f"总奖励: {episode_stats['total_reward']:.2f}")
#                     print(f"平均奖励: {episode_stats['avg_reward']:.2f}")
#                     print(f"回合时长: {episode_stats['episode_duration']:.2f}s")
                    
#                     # 打印每个智能体的统计信息
#                     for agent_id, stats in episode_stats['agent_stats'].items():
#                         print(f"  {agent_id}: 奖励={stats['total_reward']:.2f}, 步数={stats['steps']}")
                        
#             except Exception as e:
#                 print(f"回合 {episode + 1} 执行失败: {str(e)}")
#                 if verbose:
#                     import traceback
#                     traceback.print_exc()
#                 continue
        
#         return all_episode_stats
    
#     def save_agents(self, save_dir: str):
#         """保存所有智能体模型"""
#         try:
#             os.makedirs(save_dir, exist_ok=True)
            
#             for agent_id, agent in self.agents.items():
#                 filepath = os.path.join(save_dir, f"{agent_id}_model.json")
#                 agent.save_model(filepath)
#                 print(f"智能体 {agent_id} 模型已保存到 {filepath}")
                
#         except Exception as e:
#             print(f"保存模型时出错: {str(e)}")
#             raise
    
#     def get_environment_info(self) -> Dict[str, Any]:
#         """获取环境信息"""
#         if self.env is None:
#             return {}
        
#         try:
#             # 确保环境已经重置
#             try:
#                 current_agents = self.env.agents
#             except AttributeError:
#                 # 如果还没有重置，先重置一下
#                 self.env.reset()
#                 current_agents = self.env.agents
            
#             return {
#                 "possible_agents": self.env.possible_agents,
#                 "current_agents": current_agents,
#                 "observation_spaces": {agent: str(self.env.observation_space(agent)) 
#                                      for agent in self.env.possible_agents},
#                 "action_spaces": {agent: str(self.env.action_space(agent)) 
#                                 for agent in self.env.possible_agents},
#                 "environment_config": self.env_config.to_dict()
#             }
#         except Exception as e:
#             print(f"获取环境信息时出错: {str(e)}")
#             return {"error": str(e)}


# def demo_random_agents():
#     """演示随机智能体框架"""
#     print("=" * 60)
#     print("多智能体随机算法演示")
#     print("=" * 60)
    
#     try:
#         # 1. 创建环境配置
#         env_config = EnvironmentConfig(
#             n_pursuers=3,
#             n_evaders=2,
#             n_poisons=8,
#             max_cycles=200,
#             render_mode=None  # 明确设置为None
#         )
        
#         # 2. 创建环境管理器
#         print("正在创建环境...")
#         ma_env = MultiAgentEnvironment(env_config)
#         ma_env.create_environment()
        
#         # 3. 打印环境信息
#         print("获取环境信息...")
#         env_info = ma_env.get_environment_info()
        
#         if "error" in env_info:
#             print(f"获取环境信息失败: {env_info['error']}")
#             return
            
#         print(f"环境智能体: {env_info['possible_agents']}")
#         print(f"观察空间: {list(env_info['observation_spaces'].values())[0]}")
#         print(f"动作空间: {list(env_info['action_spaces'].values())[0]}")
        
#         # 4. 为每个智能体配置不同的算法
#         agent_configs = [
#             # 智能体0: 完全随机
#             AgentConfig(
#                 agent_id="pursuer_0",
#                 algorithm="random",
#                 hyperparameters={"random_seed": 42}
#             ),
#             # 智能体1: 约束随机（偏向向前移动）
#             AgentConfig(
#                 agent_id="pursuer_1", 
#                 algorithm="constrained_random",
#                 hyperparameters={
#                     "action_scale": 0.8,
#                     "bias": [0.3, 0.0],  # X方向偏置
#                     "random_seed": 123
#                 }
#             ),
#             # 智能体2: 约束随机（较小的动作）
#             AgentConfig(
#                 agent_id="pursuer_2",
#                 algorithm="constrained_random", 
#                 hyperparameters={
#                     "action_scale": 0.5,
#                     "bias": [0.0, 0.0],
#                     "random_seed": 456
#                 }
#             )
#         ]
        
#         # 5. 添加智能体
#         print("添加智能体...")
#         for config in agent_configs:
#             ma_env.add_agent(config)
        
#         # 6. 运行多个回合
#         print(f"\n可用算法: {AgentFactory.get_available_algorithms()}")
#         print("开始运行回合...")
        
#         stats = ma_env.run_multiple_episodes(
#             num_episodes=3,  # 减少回合数以便调试
#             max_steps_per_episode=50,  # 减少步数以便调试
#             render=False,
#             verbose=True
#         )
        
#         if not stats:
#             print("没有成功运行任何回合")
#             return
        
#         # 7. 统计分析
#         print("\n" + "=" * 60)
#         print("统计分析")
#         print("=" * 60)
        
#         total_rewards = [episode['total_reward'] for episode in stats]
#         episode_lengths = [episode['episode_steps'] for episode in stats]
        
#         print(f"平均总奖励: {np.mean(total_rewards):.2f} ± {np.std(total_rewards):.2f}")
#         print(f"平均回合长度: {np.mean(episode_lengths):.1f} ± {np.std(episode_lengths):.1f}")
        
#         # 每个智能体的平均表现
#         agent_rewards = {}
#         for agent_id in ma_env.agents.keys():
#             rewards = [episode['episode_rewards'][agent_id] for episode in stats]
#             agent_rewards[agent_id] = {
#                 "mean": np.mean(rewards),
#                 "std": np.std(rewards)
#             }
        
#         print("\n智能体平均表现:")
#         for agent_id, reward_stats in agent_rewards.items():
#             config = ma_env.agent_configs[agent_id]
#             print(f"  {agent_id} ({config.algorithm}): {reward_stats['mean']:.2f} ± {reward_stats['std']:.2f}")
        
#         # 8. 保存模型
#         print("\n保存模型...")
#         ma_env.save_agents("./saved_models")
        
#         print("\n演示完成!")
        
#     except Exception as e:
#         print(f"演示过程中出现错误: {str(e)}")
#         print("错误类型:", type(e).__name__)
#         import traceback
#         traceback.print_exc()


# def test_environment_only():
#     """仅测试环境创建"""
#     print("测试环境创建...")
#     try:
#         env = waterworld_v4.env(n_pursuers=2, n_evaders=2, n_poisons=5)
#         print("原始环境创建成功")
        
#         env = ss.flatten_v0(env)
#         print("扁平化成功")
        
#         env.reset()
#         print("环境重置成功")
        
#         print(f"智能体: {env.possible_agents}")
#         print(f"观察空间: {env.observation_space(env.possible_agents[0])}")
#         print(f"动作空间: {env.action_space(env.possible_agents[0])}")
        
#         env.close()
#         print("环境测试完成")
        
#     except Exception as e:
#         print(f"环境测试失败: {str(e)}")
#         import traceback
#         traceback.print_exc()


# if __name__ == "__main__":
#     # 先测试环境
#     test_environment_only()
#     print("\n" + "="*60 + "\n")
    
#     # 再运行完整演示
#     demo_random_agents()

In [15]:
from pettingzoo.butterfly import pistonball_v6

from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.sac import SACConfig
from ray.rllib.algorithms.dqn.dqn import DQNConfig
from ray.rllib.env.wrappers.pettingzoo_env import PettingZooEnv
from ray.tune.registry import register_env

register_env(
    "pistonball",
    lambda cfg: PettingZooEnv(pistonball_v6.env(num_floors=cfg.get("n_pistons", 20))),
)

config = (
    SACConfig()
    .environment("pistonball", env_config={"n_pistons": 30})
)

2025-07-16 19:13:11,375	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [16]:
import sys

sys.argv = [
    'notebook_script.py',
    '--enable-new-api-stack',
    '--num-agents=10',
    # 新增参数用于指定predator和prey的数量
    # '--n-predators=5',
    '--wandb-key=fdd7656f474bba144dea1887bcdab534bc7df647',
    '--wandb-project=waterworld-v4',
    # '--n-preys=5', 
    '--checkpoint-at-end',
    '--stop-reward=200.0',
    '--checkpoint-freq=1',
]

In [ ]:
# """Runs the PettingZoo Waterworld multi-agent env in RLlib using single policy learning.

# Other than the `pettingzoo_independent_learning.py` example (in this same folder),
# this example simply trains a single policy (shared by all agents).

# See: https://pettingzoo.farama.org/environments/sisl/waterworld/
# for more details on the environment.


# How to run this script
# ----------------------
# `python [script file name].py --enable-new-api-stack --num-agents=2`

# Control the number of agents and policies (RLModules) via --num-agents and
# --num-policies.

# This works with hundreds of agents and policies, but note that initializing
# many policies might take some time.

# For debugging, use the following additional command line options
# `--no-tune --num-env-runners=0`
# which should allow you to set breakpoints anywhere in the RLlib code and
# have the execution stop there for inspection and debugging.

# For logging to your WandB account, use:
# `--wandb-key=[your WandB API key] --wandb-project=[some project name]
# --wandb-run-name=[optional: WandB run name (within the defined project)]`


# Results to expect
# -----------------
# The above options can reach a combined reward of roughly ~0.0 after about 500k-1M env
# timesteps. Keep in mind, though, that in this setup, the agents do not have the
# opportunity to benefit from or even out other agents' mistakes (and behavior in general)
# as everyone is using the same policy. Hence, this example learns a more generic policy,
# which might be less specialized to certain "niche exploitation opportunities" inside
# the env:

# +---------------------+----------+-----------------+--------+-----------------+
# | Trial name          | status   | loc             |   iter |  total time (s) |
# |---------------------+----------+-----------------+--------+-----------------+
# | PPO_env_91f49_00000 | RUNNING  | 127.0.0.1:63676 |    200 |         605.176 |
# +---------------------+----------+-----------------+--------+-----------------+

# +--------+-------------------+-------------+
# |     ts |   combined reward |   reward p0 |
# +--------+-------------------+-------------|
# | 800000 |          0.323752 |    0.161876 |
# +--------+-------------------+-------------+
# """
# from pettingzoo.sisl import waterworld_v4

# from ray.rllib.core.rl_module.multi_rl_module import MultiRLModuleSpec
# from ray.rllib.core.rl_module.rl_module import RLModuleSpec
# from ray.rllib.env.wrappers.pettingzoo_env import PettingZooEnv
# from ray.rllib.utils.test_utils import (
#     add_rllib_example_script_args,
#     run_rllib_example_script_experiment,
# )
# from ray.tune.registry import get_trainable_cls, register_env


# parser = add_rllib_example_script_args(
#     default_iters=200,
#     default_timesteps=1000000,
#     default_reward=0.0,
# )


# if __name__ == "__main__":
#     args = parser.parse_args()

#     assert args.num_agents > 0, "Must set --num-agents > 0 when running this script!"
#     assert (
#         args.enable_new_api_stack
#     ), "Must set --enable-new-api-stack when running this script!"

#     # Here, we use the "Agent Environment Cycle" (AEC) PettingZoo environment type.
#     # For a "Parallel" environment example, see the rock paper scissors examples
#     # in this same repository folder.
#     register_env("env", lambda _: PettingZooEnv(waterworld_v4.env()))

#     base_config = (
#         get_trainable_cls(args.algo)
#         .get_default_config()
#         .environment("env")
#         .multi_agent(
#             policies={"p0"},
#             # All agents map to the exact same policy.
#             policy_mapping_fn=(lambda aid, *args, **kwargs: "p0"),
#         )
#         .training(
#             model={
#                 "vf_share_layers": True,
#             },
#             vf_loss_coeff=0.005,
#         )
#         .rl_module(
#             rl_module_spec=MultiRLModuleSpec(
#                 rl_module_specs={"p0": RLModuleSpec()},
#             ),
#         )
#     )

#     run_rllib_example_script_experiment(base_config, args)

/home/qrbao/anaconda3/envs/rllib_o/lib/python3.10/site-packages/gymnasium/spaces/box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/home/qrbao/anaconda3/envs/rllib_o/lib/python3.10/site-packages/gymnasium/spaces/box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(
/home/qrbao/anaconda3/envs/rllib_o/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:134: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/home/qrbao/anaconda3/envs/rllib_o/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
2025-07-16 19:13:13,188	INFO worker.py:1917 -- Started a local Ra

== Status ==
Current time: 2025-07-16 19:13:13 (running for 00:00:00.17)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/24 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-07-16_19-13-11_958744_3203195/artifacts/2025-07-16_19-13-13/PPO_2025-07-16_19-13-13/driver_artifacts
Number of trials: 1/1 (1 PENDING)
+---------------------+----------+-------+
| Trial name          | status   | loc   |
|---------------------+----------+-------|
| PPO_env_7bb81_00000 | PENDING  |       |
+---------------------+----------+-------+




(PPO pid=3205621) 2025-07-16 19:13:15,922	WARNING algorithm_config.py:5014 -- You are running PPO on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
(MultiAgentEnvRunner pid=3205713) 2025-07-16 19:13:18,144	WARNING deprecation.py:50 -- DeprecationWarning: `RLModule(config=[RLModuleConfig object])` has been deprecated. Use `RLModule(observation_space=.., action_space=.., inference_only=.., model_config=.., catalog_class=..)` instead. This will raise an error in the future!
(PPO pid=3205621) Install gputil for GPU system monitoring.


== Status ==
Current time: 2025-07-16 19:13:18 (running for 00:00:05.26)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/24 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-07-16_19-13-11_958744_3203195/artifacts/2025-07-16_19-13-13/PPO_2025-07-16_19-13-13/driver_artifacts
Number of trials: 1/1 (1 RUNNING)
+---------------------+----------+----------------------+
| Trial name          | status   | loc                  |
|---------------------+----------+----------------------|
| PPO_env_7bb81_00000 | RUNNING  | 192.168.0.25:3205621 |
+---------------------+----------+----------------------+




(_WandbLoggingActor pid=3205878) wandb: WARNING `start_method` is deprecated and will be removed in a future version of wandb. This setting is currently non-functional and safely ignored.
(_WandbLoggingActor pid=3205878) wandb: Currently logged in as: bqr010817 (bqr010817-kyushu-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
(_WandbLoggingActor pid=3205878) wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.
(_WandbLoggingActor pid=3205878) wandb: Tracking run with wandb version 0.20.1
(_WandbLoggingActor pid=3205878) wandb: Run data is saved locally in /tmp/ray/session_2025-07-16_19-13-11_958744_3203195/artifacts/2025-07-16_19-13-13/PPO_2025-07-16_19-13-13/driver_artifacts/PPO_env_7bb81_00000_0_2025-07-16_19-13-13/wandb/run-20250716_191321-7bb81_00000
(_WandbLoggingActor pid=3205878) wandb: Run `wandb offline` to turn off syncing.
(_WandbLoggingActor pid=3205878) wandb: Syncing run PPO_

...

(PPO(env=env; env-runners=2; learners=0; multi-agent=True) pid=3205621) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/qrbao/ray_results/PPO_2025-07-16_19-13-13/PPO_env_7bb81_00000_0_2025-07-16_19-13-13/checkpoint_000009)
(_WandbLoggingActor pid=3205878) wandb: Adding directory to artifact (/home/qrbao/ray_results/PPO_2025-07-16_19-13-13/PPO_env_7bb81_00000_0_2025-07-16_19-13-13/checkpoint_000009)... Done. 0.0s


== Status ==
Current time: 2025-07-16 19:14:19 (running for 00:01:05.78)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/24 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-07-16_19-13-11_958744_3203195/artifacts/2025-07-16_19-13-13/PPO_2025-07-16_19-13-13/driver_artifacts
Number of trials: 1/1 (1 RUNNING)
+---------------------+----------+----------------------+--------+------------------+-------+-------------------+-------------+
| Trial name          | status   | loc                  |   iter |   total time (s) |    ts |   combined return |   return p0 |
|---------------------+----------+----------------------+--------+------------------+-------+-------------------+-------------|
| PPO_env_7bb81_00000 | RUNNING  | 192.168.0.25:3205621 |     10 |          53.8968 | 40000 |          -267.065 |    -192.082 |
+---------------------+----------+----------------------+--------+------------------+-------+-------------------+-------------+




(PPO(env=env; env-runners=2; learners=0; multi-agent=True) pid=3205621) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/qrbao/ray_results/PPO_2025-07-16_19-13-13/PPO_env_7bb81_00000_0_2025-07-16_19-13-13/checkpoint_000010)
(_WandbLoggingActor pid=3205878) wandb: Adding directory to artifact (/home/qrbao/ray_results/PPO_2025-07-16_19-13-13/PPO_env_7bb81_00000_0_2025-07-16_19-13-13/checkpoint_000010)... Done. 0.0s


== Status ==
Current time: 2025-07-16 19:14:24 (running for 00:01:10.80)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/24 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-07-16_19-13-11_958744_3203195/artifacts/2025-07-16_19-13-13/PPO_2025-07-16_19-13-13/driver_artifacts
Number of trials: 1/1 (1 RUNNING)
+---------------------+----------+----------------------+--------+------------------+-------+-------------------+-------------+
| Trial name          | status   | loc                  |   iter |   total time (s) |    ts |   combined return |   return p0 |
|---------------------+----------+----------------------+--------+------------------+-------+-------------------+-------------|
| PPO_env_7bb81_00000 | RUNNING  | 192.168.0.25:3205621 |     11 |          59.1362 | 44000 |          -267.866 |    -192.811 |
+---------------------+----------+----------------------+--------+------------------+-------+-------------------+-------------+




(PPO(env=env; env-runners=2; learners=0; multi-agent=True) pid=3205621) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/qrbao/ray_results/PPO_2025-07-16_19-13-13/PPO_env_7bb81_00000_0_2025-07-16_19-13-13/checkpoint_000011)
(_WandbLoggingActor pid=3205878) wandb: Adding directory to artifact (/home/qrbao/ray_results/PPO_2025-07-16_19-13-13/PPO_env_7bb81_00000_0_2025-07-16_19-13-13/checkpoint_000011)... Done. 0.0s


In [ ]:
from torch import nn

from ray.rllib.algorithms.sac.sac import SACConfig
from ray.rllib.core.rl_module.default_model_config import DefaultModelConfig
from ray.rllib.utils.test_utils import add_rllib_example_script_args

parser = add_rllib_example_script_args(
    default_timesteps=1000000,
    default_reward=12000.0,
    default_iters=2000,
)
parser.set_defaults(enable_new_api_stack=True)
# Use `parser` to add your own custom command line options to this script
# and (if needed) use their values to set up `config` below.
args = parser.parse_args()

config = (
    SACConfig()
    .environment("HalfCheetah-v4")
    .training(
        initial_alpha=1.001,
        # lr=0.0006 is very high, w/ 4 GPUs -> 0.0012
        # Might want to lower it for better stability, but it does learn well.
        actor_lr=2e-4 * (args.num_learners or 1) ** 0.5,
        critic_lr=8e-4 * (args.num_learners or 1) ** 0.5,
        alpha_lr=9e-4 * (args.num_learners or 1) ** 0.5,
        lr=None,
        target_entropy="auto",
        n_step=(1, 5),  # 1?
        tau=0.005,
        train_batch_size_per_learner=256,
        target_network_update_freq=1,
        replay_buffer_config={
            "type": "PrioritizedEpisodeReplayBuffer",
            "capacity": 100000,
            "alpha": 0.6,
            "beta": 0.4,
        },
        num_steps_sampled_before_learning_starts=10000,
    )
    .rl_module(
        model_config=DefaultModelConfig(
            fcnet_hiddens=[256, 256],
            fcnet_activation="relu",
            fcnet_kernel_initializer=nn.init.xavier_uniform_,
            head_fcnet_hiddens=[],
            head_fcnet_activation=None,
            head_fcnet_kernel_initializer="orthogonal_",
            head_fcnet_kernel_initializer_kwargs={"gain": 0.01},
        ),
    )
    .reporting(
        metrics_num_episodes_for_smoothing=5,
        min_sample_timesteps_per_iteration=1000,
    )
)


if __name__ == "__main__":
    from ray.rllib.utils.test_utils import run_rllib_example_script_experiment

    run_rllib_example_script_experiment(config, args)

/home/qrbao/anaconda3/envs/rllib_o/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-16 15:43:35,478	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-07-16 15:43:35,709	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
usage: ipykernel_launcher.py [-h] [--algo ALGO] [--enable-new-api-stack]
                             [--framework {tf,tf2,torch}] [--env ENV]
                             [--num-env-runners NUM_ENV_RUNNERS]
                             [--num-envs-per-env-runner NUM_ENVS_PER_ENV_RUNNER]
                             [--num-agents NUM_AGENTS]
                             [--evaluation-num-

SystemExit: 2

/home/qrbao/anaconda3/envs/rllib_o/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
"""Uses Stable-Baselines3 to train agents to play the Waterworld environment using SuperSuit vector envs.

For more information, see https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html

Author: Elliot (https://github.com/elliottower)
"""
from __future__ import annotations

import glob
import os
import time

import supersuit as ss
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from pettingzoo.utils.conversions import aec_to_parallel
from pettingzoo.sisl import waterworld_v4


def train_butterfly_supersuit(
    env_fn, steps: int = 10_000, seed: int | None = 0, **env_kwargs
):
    # Train a single model to play as each agent in a cooperative Parallel environment
    env = aec_to_parallel(env_fn)
    env.reset(seed=seed)

    print(f"Starting training on {str(env.metadata['name'])}.")

    env = ss.pettingzoo_env_to_vec_env_v1(env)
    # env = ss.concat_vec_envs_v1(env, 8, num_cpus=2, base_class="stable_baselines3")

    # Note: Waterworld's observation space is discrete (242,) so we use an MLP policy rather than CNN
    model = PPO(
        MlpPolicy,
        env,
        verbose=3,
        learning_rate=1e-3,
        batch_size=256,
    )

    model.learn(total_timesteps=steps)

    model.save(f"{env.unwrapped.metadata.get('name')}_{time.strftime('%Y%m%d-%H%M%S')}")

    print("Model has been saved.")

    print(f"Finished training on {str(env.unwrapped.metadata['name'])}.")

    env.close()


def eval(env_fn, num_games: int = 100, render_mode: str | None = None, **env_kwargs):
    # Evaluate a trained agent vs a random agent
    env = env_fn.env(render_mode=render_mode, **env_kwargs)

    print(
        f"\nStarting evaluation on {str(env.metadata['name'])} (num_games={num_games}, render_mode={render_mode})"
    )

    try:
        latest_policy = max(
            glob.glob(f"{env.metadata['name']}*.zip"), key=os.path.getctime
        )
    except ValueError:
        print("Policy not found.")
        exit(0)

    model = PPO.load(latest_policy)

    rewards = {agent: 0 for agent in env.possible_agents}

    # Note: We train using the Parallel API but evaluate using the AEC API
    # SB3 models are designed for single-agent settings, we get around this by using he same model for every agent
    for i in range(num_games):
        env.reset(seed=i)

        for agent in env.agent_iter():
            obs, reward, termination, truncation, info = env.last()

            for a in env.agents:
                rewards[a] += env.rewards[a]
            if termination or truncation:
                break
            else:
                act = model.predict(obs, deterministic=True)[0]

            env.step(act)
    env.close()

    avg_reward = sum(rewards.values()) / len(rewards.values())
    print("Rewards: ", rewards)
    print(f"Avg reward: {avg_reward}")
    return avg_reward


if __name__ == "__main__":
    agent_algos = ["PPO", "PPO", "DQN", "DQN","PPO", "PPO", "DQN", "DQN","DQN", "DQN"]
    env_fn = waterworld_v4.env(
        render_mode="human",
        n_predators=5,
        n_preys=5,
        n_evaders=1,
        n_obstacles=2,
        obstacle_coord=[(0.2, 0.2), (0.8, 0.2)],
        n_poisons=20,
        agent_algorithms=agent_algos
    )
    env_kwargs = {}


In [3]:
    print(123)
    # Train a model (takes ~3 minutes on GPU)
    train_butterfly_supersuit(env_fn, steps=196_608, seed=0, **env_kwargs)

    # Evaluate 10 games (average reward should be positive but can vary significantly)
    eval(env_fn, num_games=10, render_mode=None, **env_kwargs)

    # Watch 2 games
    eval(env_fn, num_games=2, render_mode="human", **env_kwargs)

123
Reset complete:
  Agents: ['predator_0', 'predator_1', 'predator_2', 'predator_3', 'predator_4', 'prey_0', 'prey_1', 'prey_2', 'prey_3', 'prey_4']
  _cumulative_rewards keys: ['predator_0', 'predator_1', 'predator_2', 'predator_3', 'predator_4', 'prey_0', 'prey_1', 'prey_2', 'prey_3', 'prey_4']
Starting training on waterworld_v4.
Using cuda device


ValueError: The environment is of type <class 'supersuit.vector.markov_vector_wrapper.MarkovVectorEnv'>, not a Gymnasium environment. In this case, we expect OpenAI Gym to be installed and the environment to be an OpenAI Gym environment.

In [8]:

env_fn = waterworld_v4.env()
env = aec_to_parallel(env_fn)